## 파일을 실행 시키기에 앞서
1. Colab 이 아닌 jupyter notebook 으로 실행시켜주세요. (이미지가 깨집니다.)  
2. 실행하면 크롤링 시간이 오래걸리니 결과 위주로 봐주세요

# 웹 크롤링 / 5조 송준영, 이동언, 이지현, 주현우, 하동호

## 대상 선정 (웹툰)

- 1. 주제, 목적 (웹툰)  
    - 최근 OTT 서비스에서 제공하는 영화 및 드라마 사이에, 웹툰을 원작으로 한 영화들이 좋은 성과를 거두고 있음  
      ( 지금 우리 학교는, 지옥, 스위트홈, DP 등 )  
      영화/드라마화 하기 위한 웹툰을 선택하기 위해 웹툰과 관련된 정보들을 크롤링 해보기로 함
    
    
- 2. 요구사항 정의  
    - 웹툰 종류 및 분류, 댓글 모음, 별점, 완결 여부 및 연재 여부, 최근 연재일, 조회수  
      연재 플랫폼, 작가, 장르, 연령제한, 유료/무료, 같은 작가의 다른 작품, 작가 정보 등
    - 가능 / 불가능, 불가능의 대안 (조회수 → 좋아요)
      
      
- 3. 수집대상 선정
    - 네이버 웹툰, 카카오 웹툰

## 수집대상 분석

* 웹사이트 분석
    - 네이버 웹툰 (정적 페이지)
    - 카카오 웹툰 (동적 페이지)
* 네이버 검색 (인물 사전)

## 설계

* 1. 데이터 흐름 / 엔티티 관계  
  '작가' 테이블은 '웹툰 플랫폼' 테이블이 아닌, '네이버 웹툰' 테이블과 연결되어 있음 (이미지 수정 필요)
![image-4.png](attachment:image-4.png)

* 2. 테이블 정의  
    * 작가 테이블의 경우 컬럼이 생성되는 함수가 있어서 정의테이블을 정리하지 못했음.
    * 그 외, 같은 코드, 같은 방식으로 생성되는 테이블의 경우는 이번 프로젝트 진행할때 생략하고 진행하였음  
      (다양한 종류의 코딩을 해보기 위함)
* 네이버 메인
![image-2.png](attachment:image-2.png)
* 카카오
![image-3.png](attachment:image-3.png)

# 개발 및 테스트

## 네이버 웹툰 (메인 테이블)
![image.png](attachment:image.png)

* Import 모듈

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import datetime as dt
from selenium.webdriver.common.by import By
import psycopg2

In [ ]:
url = 'https://comic.naver.com/webtoon/weekday'
response = requests.get(url)
response.raise_for_status()
html=response.content
soup=BeautifulSoup(html, 'html.parser')

# # col - 요일별 # select는 정확하게 집어야 하나?
# divs=soup.select('div.list_area.daily_all > div.col > div.col_inner')
# print(divs)

* DB 연결

In [ ]:
# !pip install psycopg2

In [ ]:
import psycopg2

try:
    conn = psycopg2.connect(host="localhost",database="220205", user="postgres", password="admin",port=5432)
    print('===접속 성공===')
    cur = conn.cursor()
except psycopg2.DatabaseError as db_err:
    print('접속 오류 !!')
    print(db_err)


===접속 성공===


### 데이터 테이블 생성

* 웹툰 테이블 생성  
    id(키값), date(연재 요일/완결), title(웹툰 제목),  
    link(해당 웹툰 링크), writer(작가), platform(플랫폼) / 총 6개 컬럼

In [ ]:
cur = conn.cursor()
try:
    cur.execute('CREATE TABLE if not exists webtoon (id serial, date varchar(50), title varchar(200), link varchar(500), writer varchar(200), platform varchar(200));')
    conn.commit()
    print('success')
except:
    print('fail')


success


* 위 이미지에서 제목에 해당하는 부분을 찾아 해당 정보 크롤링 + DB 저장

In [ ]:
cur = conn.cursor()

divs=soup.find_all('div',{'class' : 'col_inner'})
# print(divs)
fail_db_insert = 0

platform='네이버'
for div in divs: # 7개 요일별로
#     print(div)
#     print('-'*70)
    h4 =div.find('h4')
    when_toon=h4.text[0]    #// 요일명 저장
    yo1=div.find_all('li') # 작품별로
    for yo in yo1:
        links=yo.find_all('a')
#         print(links[1]['href'])
#         print(links[1]['title'])
#         print('-'*70)
        link='https://comic.naver.com'+links[1]['href'][:28] #// 주소 저장 # 뒤에 요일 빼도 들어가짐
        title=links[1]['title'] #// 이름 저장

        
        # DB에 넣기
        
        try:
            cur.execute('INSERT INTO webtoon (date, title, link, platform) values (%s, %s, %s, %s)',
                       (when_toon, title, link, platform))
            conn.commit()
            print('succss')
        except:
            cur.rollback()
            fail_db_insert += 1
            print('fail')
        
            

print(str(fail_db_insert)+' 회 실패')


succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss
succss

#### 중복값 정리
* 웹툰 테이블에서 "제목 기준" 중복 웹툰 찾아서 출력
* 정리 후에 다시 실행하는 바람에 결과값은 빈 리스트

In [ ]:
# 중복 찾기
cur = conn.cursor()
try:
    cur.execute('select title from webtoon group by title having count (title) > 1')
    joongbok = cur.fetchall()
    conn.commit()
    print('succss')
    print(joongbok) # 중복값 추출
except:
    conn.rollback()
    print('fail')


succss
[]


* 중복되어 있는 웹툰들이 다른 요일(date) 값을 가지므로, 합쳐서 UPDATE  
    '월' / '화' 의 데이터 2개 → '월, 화' 의 데이터 2개

In [ ]:
# 요일 정리
cur = conn.cursor()
for title_name in joongbok:
    
    try:
        cur.execute("select * from webtoon where title = '%s'" %title_name[0])
        joongbok_toon = cur.fetchall()
        conn.commit()
#         print(joongbok_toon)

        jb_date = []
        for jb_toon_date in joongbok_toon:
            jb_date.append(jb_toon_date[1])
        jb_date_update = ', '.join(jb_date)
        cur.execute("update webtoon set date=%s where title=%s",
                    (jb_date_update, title_name[0]))
        conn.commit()
#         print('succss')
        
    except:
        print('fail')
        


* 중복으로 저장된 웹툰은 어떤 것이 있을까?

In [ ]:
print(joongbok)

[('먹는 인생',), ('신림/남/22',), ('윌유메리미',), ('조조코믹스',), ('삼국지톡',), ('독립일기',), ('소극적 인간',), ('쇼미더럭키짱!',), ('2022 서브병에 빠지다!',), ('로어 올림푸스',), ('니나의 마법서랍',), ('여고생 드래곤',)]


* 중복 웹툰 중 id 값이 가장 작은 것은 제외하고,  
  남은 중복 웹툰들의 id 값을 찾음

In [ ]:
# 중복 웹툰 id 제일 작은것 빼고 
cur = conn.cursor()
try:
    cur.execute('select webtoon.id from (SELECT ROW_NUMBER() OVER (PARTITION BY title ORDER BY id) AS NUM, * FROM webtoon) as A, webtoon where webtoon.id = A.id and A.num >=2;')
    jb_id=cur.fetchall()
    conn.commit()
    print('succss')
    print(jb_id)
except:
    print('fail')

succss
[]


* 위에서 찾은 id 값을 갖는 것을 다 삭제함

In [ ]:
# 중복 날리기
for del_id in jb_id:
    try:
        cur.execute("delete from webtoon where id = '%s'" %del_id[0])
        conn.commit()
        print('success')
    except:
        print('fail')

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


* 중복 삭제 되었는지 한번 확인

In [ ]:
# 중복값 빠진지 확인 # 중복확인
cur = conn.cursor()
try:
    cur.execute('select title from webtoon group by title having count (title) > 1')
    joongbok = cur.fetchall()
    conn.commit()
    print('succss')
    print(joongbok) # 중복값 추출
except:
    conn.rollback()
    print('fail')

### 네이버 작가 + 장르 + 나이(연령제한)

![image.png](attachment:image.png)

In [ ]:
# 좋아요 컬럼 생성 (like_it)
try:
    cur.execute('alter table webtoon add column like_it varchar(100);')
    conn.commit()
    print('success')
except:
    conn.rollback()
    print('fail')

In [ ]:
# 장르, 연령제한 컬럼 추가 (genre, age)
cur = conn.cursor()
try:
    cur.execute('alter table webtoon add column genre varchar(200);')
    cur.execute('alter table webtoon add column age varchar(200);')
    conn.commit()
    print('succss')
except:
    conn.rollback()
    print('fail')

succss


* 2.1.1 에서 저장했던 link 들을 가져옴. (url 정해주기 위해서, DB에 아까 크롤링한 정보를 사용하는 것)

In [ ]:
# 링크 추출 (웹툰 안에 들어가야 정보가 있기 때문)
cur = conn.cursor()
try:
    cur.execute('select link from webtoon')
    webtoon_links= cur.fetchall()
    conn.commit()
    print('succss')
#     print(webtoon_links) # 중복값 추출
except:
    conn.rollback()
    print('fail')

print(len(webtoon_links))

succss
1638


In [ ]:
# 작가, 장르, 연령제한 넣어주기
loading = 0
for webtoon_link in webtoon_links:
    url=webtoon_link[0]
    response = requests.get(url)
    response.raise_for_status()
    html=response.content
    soup=BeautifulSoup(html, 'html.parser')
    
    divs=soup.find('div',{'class' : 'detail'})
    div=divs.find('span',{'class' : 'wrt_nm'})
    writer_name = div.text.strip()
#     print(writer_name)
    
    divs2=soup.find('p',{'class':'detail_info'})
    genre=divs2.find('span',{'class':'genre'}).text
    try:
        age=divs2.find('span',{'class':'age'}).text
    except AttributeError:
        print('age 없음')
        age=''
    
    
    try:
        cur.execute('update webtoon set (writer, genre, age) = (%s, %s, %s) where link=%s',
                   (writer_name, genre, age, webtoon_link[0]))
        conn.commit()
        loading += 1
        print('success\t'+str(loading)+' / '+str(len(webtoon_links)))
    except:
        conn.rollback()
        print('fail')


success	1 / 1638
success	2 / 1638
success	3 / 1638
success	4 / 1638
success	5 / 1638
success	6 / 1638
success	7 / 1638
success	8 / 1638
success	9 / 1638
success	10 / 1638
success	11 / 1638
success	12 / 1638
success	13 / 1638
success	14 / 1638
success	15 / 1638
success	16 / 1638
success	17 / 1638
success	18 / 1638
success	19 / 1638
success	20 / 1638
success	21 / 1638
success	22 / 1638
success	23 / 1638
success	24 / 1638
success	25 / 1638
success	26 / 1638
success	27 / 1638
success	28 / 1638
success	29 / 1638
success	30 / 1638
success	31 / 1638
success	32 / 1638
success	33 / 1638
success	34 / 1638
success	35 / 1638
success	36 / 1638
success	37 / 1638
success	38 / 1638
success	39 / 1638
success	40 / 1638
success	41 / 1638
success	42 / 1638
success	43 / 1638
success	44 / 1638
success	45 / 1638
success	46 / 1638
success	47 / 1638
success	48 / 1638
success	49 / 1638
success	50 / 1638
success	51 / 1638
success	52 / 1638
success	53 / 1638
success	54 / 1638
success	55 / 1638
success	56 / 1638
s

success	437 / 1638
success	438 / 1638
age 없음
success	439 / 1638
success	440 / 1638
success	441 / 1638
success	442 / 1638
success	443 / 1638
success	444 / 1638
success	445 / 1638
age 없음
success	446 / 1638
age 없음
success	447 / 1638
success	448 / 1638
success	449 / 1638
success	450 / 1638
age 없음
success	451 / 1638
age 없음
success	452 / 1638
success	453 / 1638
success	454 / 1638
success	455 / 1638
success	456 / 1638
success	457 / 1638
success	458 / 1638
success	459 / 1638
success	460 / 1638
age 없음
success	461 / 1638
success	462 / 1638
success	463 / 1638
success	464 / 1638
success	465 / 1638
success	466 / 1638
success	467 / 1638
age 없음
success	468 / 1638
success	469 / 1638
success	470 / 1638
success	471 / 1638
success	472 / 1638
success	473 / 1638
age 없음
success	474 / 1638
success	475 / 1638
success	476 / 1638
success	477 / 1638
success	478 / 1638
success	479 / 1638
success	480 / 1638
age 없음
success	481 / 1638
success	482 / 1638
success	483 / 1638
success	484 / 1638
success	485 / 1638
succes

success	847 / 1638
success	848 / 1638
success	849 / 1638
success	850 / 1638
success	851 / 1638
success	852 / 1638
success	853 / 1638
success	854 / 1638
success	855 / 1638
success	856 / 1638
success	857 / 1638
success	858 / 1638
success	859 / 1638
success	860 / 1638
success	861 / 1638
success	862 / 1638
success	863 / 1638
success	864 / 1638
success	865 / 1638
success	866 / 1638
success	867 / 1638
success	868 / 1638
success	869 / 1638
success	870 / 1638
success	871 / 1638
success	872 / 1638
success	873 / 1638
success	874 / 1638
success	875 / 1638
success	876 / 1638
success	877 / 1638
success	878 / 1638
success	879 / 1638
success	880 / 1638
success	881 / 1638
success	882 / 1638
success	883 / 1638
success	884 / 1638
success	885 / 1638
success	886 / 1638
success	887 / 1638
success	888 / 1638
success	889 / 1638
success	890 / 1638
success	891 / 1638
success	892 / 1638
success	893 / 1638
success	894 / 1638
success	895 / 1638
success	896 / 1638
success	897 / 1638
success	898 / 1638
success	899 

success	1266 / 1638
success	1267 / 1638
success	1268 / 1638
success	1269 / 1638
success	1270 / 1638
success	1271 / 1638
success	1272 / 1638
success	1273 / 1638
success	1274 / 1638
success	1275 / 1638
success	1276 / 1638
success	1277 / 1638
success	1278 / 1638
success	1279 / 1638
success	1280 / 1638
success	1281 / 1638
success	1282 / 1638
success	1283 / 1638
success	1284 / 1638
success	1285 / 1638
success	1286 / 1638
success	1287 / 1638
success	1288 / 1638
success	1289 / 1638
success	1290 / 1638
success	1291 / 1638
success	1292 / 1638
success	1293 / 1638
success	1294 / 1638
success	1295 / 1638
success	1296 / 1638
success	1297 / 1638
success	1298 / 1638
success	1299 / 1638
success	1300 / 1638
success	1301 / 1638
success	1302 / 1638
success	1303 / 1638
success	1304 / 1638
success	1305 / 1638
success	1306 / 1638
success	1307 / 1638
success	1308 / 1638
success	1309 / 1638
success	1310 / 1638
success	1311 / 1638
success	1312 / 1638
success	1313 / 1638
success	1314 / 1638
success	1315 / 1638


### 연재 별점(작품별 평균)
* 전체 요일 페이지가 아닌, 요일별 연재 페이지에서 평균 별점을 확인할 수 있었음

In [ ]:
cur = conn.cursor()
try:
    cur.execute('alter table webtoon add column star_score varchar(20);')
    conn.commit()
    print('success')
except:
    conn.rollback()
    print('fail')

success


* 크롤링 후 DB 저장

![image.png](attachment:image.png)

In [ ]:
# 별점 연재 월~일 업데이트
url_list = []
url_list.append('https://comic.naver.com/webtoon/weekdayList?week=mon')
url_list.append('https://comic.naver.com/webtoon/weekdayList?week=tue')
url_list.append('https://comic.naver.com/webtoon/weekdayList?week=wed')
url_list.append('https://comic.naver.com/webtoon/weekdayList?week=thu')
url_list.append('https://comic.naver.com/webtoon/weekdayList?week=fri')
url_list.append('https://comic.naver.com/webtoon/weekdayList?week=sat')
url_list.append('https://comic.naver.com/webtoon/weekdayList?week=sun')


for url in url_list:
    response = requests.get(url)
    response.raise_for_status()
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')


    divs=soup.find('div',{'class' : 'list_area'})
    lists=divs.find_all('li')

    for lit in lists:
        title_dl = lit.find('dl')
        title_dt = title_dl.find('dt')
        title = title_dt.find('a')['title']
        links = title_dt.find('a')['href'][:28]
        link = 'https://comic.naver.com' + links
        print(title, '\t', link) # 제목, 링크

        star_div = title_dl.find('div',{'class':'rating_type'})
        star_score = star_div.find('strong').text
        print(star_score)
        print('-'*70)

        try:
            cur.execute('update webtoon set star_score = %s where link = %s',
                       (star_score, link))
            conn.commit()
            print('success')
        except:
            conn.rollback()
            print('fail')

쇼미더럭키짱! 	 https://comic.naver.com/webtoon/list?titleId=783054
9.27
----------------------------------------------------------------------
success
참교육 	 https://comic.naver.com/webtoon/list?titleId=758037
9.82
----------------------------------------------------------------------
success
신의 탑 	 https://comic.naver.com/webtoon/list?titleId=183559
9.91
----------------------------------------------------------------------
success
2022 서브병에 빠지다! 	 https://comic.naver.com/webtoon/list?titleId=787411
9.92
----------------------------------------------------------------------
success
퀘스트지상주의 	 https://comic.naver.com/webtoon/list?titleId=783052
9.58
----------------------------------------------------------------------
success
소녀의 세계 	 https://comic.naver.com/webtoon/list?titleId=654774
9.95
----------------------------------------------------------------------
success
장씨세가 호위무사 	 https://comic.naver.com/webtoon/list?titleId=728750
9.94
--------------------------------------------------------

쇼미더럭키짱! 	 https://comic.naver.com/webtoon/list?titleId=783054
9.27
----------------------------------------------------------------------
success
김부장 	 https://comic.naver.com/webtoon/list?titleId=783053
8.59
----------------------------------------------------------------------
success
여신강림 	 https://comic.naver.com/webtoon/list?titleId=703846
9.45
----------------------------------------------------------------------
success
2022 서브병에 빠지다! 	 https://comic.naver.com/webtoon/list?titleId=787411
9.92
----------------------------------------------------------------------
success
1을 줄게 	 https://comic.naver.com/webtoon/list?titleId=773522
9.96
----------------------------------------------------------------------
success
한림체육관 	 https://comic.naver.com/webtoon/list?titleId=743139
9.73
----------------------------------------------------------------------
success
내가 키운 S급들 	 https://comic.naver.com/webtoon/list?titleId=784248
9.86
-----------------------------------------------------------

쇼미더럭키짱! 	 https://comic.naver.com/webtoon/list?titleId=783054
9.27
----------------------------------------------------------------------
success
전지적 독자 시점 	 https://comic.naver.com/webtoon/list?titleId=747269
9.97
----------------------------------------------------------------------
success
화산귀환 	 https://comic.naver.com/webtoon/list?titleId=769209
9.96
----------------------------------------------------------------------
success
헬퍼 2 : 킬베로스 	 https://comic.naver.com/webtoon/list?titleId=670143
8.93
----------------------------------------------------------------------
success
조조코믹스 	 https://comic.naver.com/webtoon/list?titleId=774862
9.97
----------------------------------------------------------------------
success
먹는 인생 	 https://comic.naver.com/webtoon/list?titleId=784255
9.96
----------------------------------------------------------------------
success
캐슬 	 https://comic.naver.com/webtoon/list?titleId=736744
9.96
---------------------------------------------------------------

독립일기 	 https://comic.naver.com/webtoon/list?titleId=748105
9.97
----------------------------------------------------------------------
success
쇼미더럭키짱! 	 https://comic.naver.com/webtoon/list?titleId=783054
9.27
----------------------------------------------------------------------
success
더 복서 	 https://comic.naver.com/webtoon/list?titleId=736989
9.92
----------------------------------------------------------------------
success
연애혁명 	 https://comic.naver.com/webtoon/list?titleId=570503
9.87
----------------------------------------------------------------------
success
나노마신 	 https://comic.naver.com/webtoon/list?titleId=747271
9.93
----------------------------------------------------------------------
success
기기괴괴 	 https://comic.naver.com/webtoon/list?titleId=557672
9.90
----------------------------------------------------------------------
success
신림/남/22 	 https://comic.naver.com/webtoon/list?titleId=783055
5.53
----------------------------------------------------------------------
s

쇼미더럭키짱! 	 https://comic.naver.com/webtoon/list?titleId=783054
9.27
----------------------------------------------------------------------
success
외모지상주의 	 https://comic.naver.com/webtoon/list?titleId=641253
9.47
----------------------------------------------------------------------
success
갓 오브 하이스쿨 	 https://comic.naver.com/webtoon/list?titleId=318995
9.64
----------------------------------------------------------------------
success
내과 박원장 	 https://comic.naver.com/webtoon/list?titleId=781354
9.93
----------------------------------------------------------------------
success
2022 서브병에 빠지다! 	 https://comic.naver.com/webtoon/list?titleId=787411
9.92
----------------------------------------------------------------------
success
나 혼자 만렙 뉴비 	 https://comic.naver.com/webtoon/list?titleId=773797
9.83
----------------------------------------------------------------------
success
광마회귀 	 https://comic.naver.com/webtoon/list?titleId=776601
9.96
--------------------------------------------------

2022 서브병에 빠지다! 	 https://comic.naver.com/webtoon/list?titleId=787411
9.92
----------------------------------------------------------------------
success
프리드로우 	 https://comic.naver.com/webtoon/list?titleId=597447
9.83
----------------------------------------------------------------------
success
취사병 전설이 되다 	 https://comic.naver.com/webtoon/list?titleId=727188
9.85
----------------------------------------------------------------------
success
조조코믹스 	 https://comic.naver.com/webtoon/list?titleId=774862
9.97
----------------------------------------------------------------------
success
신림/남/22 	 https://comic.naver.com/webtoon/list?titleId=783055
5.53
----------------------------------------------------------------------
success
먹는 인생 	 https://comic.naver.com/webtoon/list?titleId=784255
9.96
----------------------------------------------------------------------
success
은탄 	 https://comic.naver.com/webtoon/list?titleId=787500
9.94
----------------------------------------------------------

독립일기 	 https://comic.naver.com/webtoon/list?titleId=748105
9.97
----------------------------------------------------------------------
success
싸움독학 	 https://comic.naver.com/webtoon/list?titleId=736277
9.83
----------------------------------------------------------------------
success
입학용병 	 https://comic.naver.com/webtoon/list?titleId=758150
9.89
----------------------------------------------------------------------
success
약한영웅 	 https://comic.naver.com/webtoon/list?titleId=710751
9.85
----------------------------------------------------------------------
success
열렙전사 	 https://comic.naver.com/webtoon/list?titleId=670152
9.93
----------------------------------------------------------------------
success
투신전생기 	 https://comic.naver.com/webtoon/list?titleId=774044
9.93
----------------------------------------------------------------------
success
존망코인 	 https://comic.naver.com/webtoon/list?titleId=783051
8.82
----------------------------------------------------------------------
succes

### 좋아요 (셀레니움)

- 좋아요 숫자 크롤링하는 부분에서 발생한 이슈  
    - requests 모듈로 u_likeit_module 이 비어 있음
    - 셀레니움을 사용해 u_likeit_module 모듈이 실행되서 가지는 값을 크롤링 시킴

* loading 으로 450번 인덱스부터 실행시켰는데,  
  크롤링 중에 에러가 발생하여 중간부터 다시 크롤링 하였음  
  에러 발생 부분 : '홀더' 라는 웹툰의 좋아요가 없음 → 숫자 0 이 아니라 버튼이름인 '좋아요'가 저장되어 값 타입이 달라서 발생  
      이 부분의 에러는 완결 웹툰의 데이터까지 넣고 재 실행 했을 때 발생한 에러임.  
      그래서 현재 like_it의 컬럼 타입이 int라서 발생한 문제.  
      다음 코드는 그 이전에 like_it 컬럼 타입을 vchar → int 로 바꿔주는 과정임. (순서 주의)  

In [ ]:
# 좋아요 (셀레니움)
driver = webdriver.Chrome()
len_webtoon_links=len(webtoon_links)
loading=450

for webtoon_link in webtoon_links[450:]:
    url=webtoon_link[0]
    driver.get(url)
    time.sleep(2)
    
    elem_title = driver.find_element_by_class_name('title')
    print(elem_title.text, end='\t')
    elem = driver.find_element_by_class_name('u_likeit_module')
    print(elem.text)
    like_it_text=elem.text.rstrip('+').replace(',','')
    if like_it_text == '좋아요':
        like_it=0
    else:
        like_it=int(like_it_text)
    
    try:
        cur.execute('update webtoon set like_it = %s where link=%s',
                   (like_it, webtoon_link[0]))
        conn.commit()
        loading += 1
        print('success\t'+str(loading)+'/'+str(len_webtoon_links))
    except:
        conn.rollback()
        print('fail')

C:\Users\q\AppData\Local\Temp/ipykernel_15868/900697230.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem_title = driver.find_element_by_class_name('title')
C:\Users\q\AppData\Local\Temp/ipykernel_15868/900697230.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_class_name('u_likeit_module')


바스키	1
success	451/1638
갤러리L : 당신의 기억을 보관해드립니다	1
success	452/1638
홀더	좋아요
success	453/1638
여고생 임연수	3
success	454/1638
왕세자 입학도	43,597
success	455/1638
손 잡아 볼래?	5,457
success	456/1638
[사전이벤트] 슈퍼캐스팅: BTS	328
success	457/1638
걸어서 30분	99,999+
success	458/1638
썸내일	816
success	459/1638
찬란하지 않아도 괜찮아, 새벽	527
success	460/1638
모기전쟁	16
success	461/1638
잉여특공대	70,229
success	462/1638
후아유!	39,699
success	463/1638
진짜 정말 맹세코 좋아해	32,533
success	464/1638
온실 속 화초	477
success	465/1638
오늘의 순정망화	99,999+
success	466/1638
아이즈	1,441
success	467/1638
트리니티 원더	11
success	468/1638
삼매경	1,522
success	469/1638
살아간다	280
success	470/1638
선의의 경쟁	99,707
success	471/1638
던전 씹어먹는 아티팩트	3,142
success	472/1638
오피스 누나 이야기	27,329
success	473/1638
리프로듀싱	2
success	474/1638
선배, 그 립스틱 바르지 마요	2,280
success	475/1638
군주	1,017
success	476/1638
헬프미	1,111
success	477/1638
연애는 전쟁!	650
success	478/1638
악인	1,266
success	479/1638
아이레	15,033
success	480/1638
분당구의 용사 지망생	좋아요
success	481/1638
파이게임	55,998
success	482/1638
호시탐탐	852
success	483/1638


이별의 품격	44
success	723/1638
차라리 만나지 않았더라면	12
success	724/1638
사냥개들	50,687
success	725/1638
옆집친구	99,999+
success	726/1638
스크립토리움	10,695
success	727/1638
나의 우주	99,999+
success	728/1638
우리 사장님은 개	75
success	729/1638
헬 인 파라다이스	19
success	730/1638
차가운 장례식	38
success	731/1638
열불 로맨스	99,999+
success	732/1638
나의 첫번째 새벽	36,924
success	733/1638
갓물주	99,999+
success	734/1638
쌈빡	493
success	735/1638
죽어도 로맨스	35
success	736/1638
극야	97,293
success	737/1638
선녀야 야옹해봐!	31,215
success	738/1638
커넥트	58,052
success	739/1638
연하의 맛	371
success	740/1638
인생존망	99,999+
success	741/1638
일진이 사나워	99,999+
success	742/1638
봉이 김선달	31
success	743/1638
잭슨의 관	187
success	744/1638
인싸라이프	84,878
success	745/1638
에리타	20,538
success	746/1638
꼬맹이를 부탁해!	50,292
success	747/1638
무모협지	31,972
success	748/1638
레사 시즌2~3	99,999+
success	749/1638
하늘아래 우체부	39
success	750/1638
라라파루자	29
success	751/1638
괴물신부	97,052
success	752/1638
언데드	99,999+
success	753/1638
화장 지워주는 남자	99,999+
success	754/1638
악마와 계약연애	99,999+
success	755/1638
땅 보고 걷는 아이	9

소유	99,999+
success	987/1638
둥굴레차!	99,999+
success	988/1638
허니허니 웨딩	99,999+
success	989/1638
12차원 소년들	44,878
success	990/1638
연애의 정령	99,999+
success	991/1638
몽연	54,073
success	992/1638
허니버니	96,929
success	993/1638
하늘쌤은 피곤해	36,165
success	994/1638
귀도호가록	99,999+
success	995/1638
밥 먹고 갈래요?	99,999+
success	996/1638
창궐	47,328
success	997/1638
조선팔도 최강아이돌	44,371
success	998/1638
하나의 하루	99,999+
success	999/1638
클로즈업	42,515
success	1000/1638
언원티드	99,999+
success	1001/1638
[드라마원작] 알고있지만	99,999+
success	1002/1638
오직 나의 주인님	99,999+
success	1003/1638
데우스 엑스 마키나	46,625
success	1004/1638
한국만화 또 다른 시선	16,714
success	1005/1638
고교생을 환불해 주세요	99,999+
success	1006/1638
미시령	99,999+
success	1007/1638
조의 영역	99,999+
success	1008/1638
왕으로 살다	42,663
success	1009/1638
파리의 우리동네	35,005
success	1010/1638
공복의 저녁식사	99,999+
success	1011/1638
꼬리잡기	883
success	1012/1638
신군	172
success	1013/1638
찌질하지만 로맨스는 하고 싶어	34
success	1014/1638
오로지 오로라	900
success	1015/1638
아는 여자애	99,999+
success	1016/1638
놓지마 정신줄 시즌2	99,999+
success	

동네변호사 조들호 시즌1	64,259
success	1234/1638
삼국전투기	27,098
success	1235/1638
손의 흔적	99,999+
success	1236/1638
2호선 세입자	64,939
success	1237/1638
후유증 2	60,712
success	1238/1638
화이트멜로우	41,561
success	1239/1638
천적	35,463
success	1240/1638
오빠 왔다	99,999+
success	1241/1638
하이브 1~2	99,999+
success	1242/1638
어빌리티	29,441
success	1243/1638
철벽! 연애 시뮬레이션	99,999+
success	1244/1638
마녀사냥	99,999+
success	1245/1638
2015 사이	99,999+
success	1246/1638
아이덴티티	63,970
success	1247/1638
오민혁 단편선	59,767
success	1248/1638
가우스전자 시즌1~2	99,999+
success	1249/1638
사또(Satto)	21,609
success	1250/1638
천국의 신화	48,155
success	1251/1638
2015 루키 단편선	51,161
success	1252/1638
스페이스 차이나드레스	41,970
success	1253/1638
나는 너를 보았다	40,888
success	1254/1638
늘 푸른 찻집	99,999+
success	1255/1638
미래소녀	65,943
success	1256/1638
리얼주주	47,543
success	1257/1638
고고고	71,357
success	1258/1638
라크리모사	41,865
success	1259/1638
데드데이즈(DEAD DAYS)	99,999+
success	1260/1638
아이소포스 1, 2부	48,644
success	1261/1638
장미아파트 공경비	62,392
success	1262/1638
공주는 잠 못 이루고	71,800
success	

우연일까?	4,234
success	1486/1638
세개의시간	5,197
success	1487/1638
[드라마원작] 지금 우리 학교는	27,439
success	1488/1638
남기한엘리트만들기	28,941
success	1489/1638
레드초콜릿	4,132
success	1490/1638
다욤이의 다이어트 다이어리	4,082
success	1491/1638
단군할배요	1,436
success	1492/1638
어서오세요.305호에	27,753
success	1493/1638
단편 할머니/나는내일죽는다	64
success	1494/1638
2011 미스테리 단편	20,341
success	1495/1638
까치우는 날	67
success	1496/1638
잉잉잉	2,663
success	1497/1638
굿모닝 스페이스	4,484
success	1498/1638
멜로홀릭	3,986
success	1499/1638
불량 뱀파이어	262
success	1500/1638
스토커	9,556
success	1501/1638
스마일 브러시, 오래된 사진.	1,474
success	1502/1638
프로젝트 X	3,240
success	1503/1638
테제	3,921
success	1504/1638
은하연인전	3,727
success	1505/1638
묵회	1,619
success	1506/1638
뷰티플 게임	32
success	1507/1638
바이올린처럼.	2,307
success	1508/1638
견우와 직녀	17,631
success	1509/1638
징글정글	462
success	1510/1638
리턴	1,039
success	1511/1638
흐드러지다	2,156
success	1512/1638
혜성같은 소년	613
success	1513/1638
핑크레이디 클래식	9,448
success	1514/1638
힘내요 일본!-[릴레이웹툰]	224
success	1515/1638
그린스마일	1,447
success	1516/1638
보톡스	633
succ

- 컬럼 타입 변경 
    - 좋아요 숫자를 str으로 받기 때문에 ',' 와 '+'를 제거해서 숫자로 바꿔줌
    - 컬럼 타입을 바꿔서 새로운 컬럼에 옮겨줌
    - 이전 컬럼 삭제 및 새로운 컬럼의 이름을 바꿔줌

In [ ]:

cur = conn.cursor()
try:
    cur.execute('alter table webtoon add column tt int;')                   # 컬럼 생성 (tt)
    cur.execute("update webtoon set like_it = replace(like_it, '+','')")    # like_it 컬럼에서 + 제거
    cur.execute("update webtoon set tt = replace(like_it, ',','')::int")    # , 제거후 tt 컬럼으로 옮겨줌
    cur.execute('alter table webtoon drop column like_it')                  # 기존 like_it 컬럼 삭제
    cur.execute('alter table webtoon rename column tt to like_it')          # tt -> like_it 이름 변경
    conn.commit()
    print('success')
except:
    conn.rollback()
    print('fail')

### 이제 order by 숫자열로 like_it 인식

success


### 네이버 완결

![image.png](attachment:image.png)

In [ ]:
url = 'https://comic.naver.com/webtoon/finish'  # 완결 웹툰 페이지
response = requests.get(url)
response.raise_for_status()
html = response.content
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
divs=soup.find('div',{'class' : 'list_area'})
lists=divs.find_all('li')
print(len(lists))

# 완결 웹툰의 개수

1146


- 완결 웹툰 업데이트 (제목, 완결, 링크, 별점, 플랫폼)

In [ ]:
platform = '네이버'

for lit in lists:
    title_dl = lit.find('dl')
    title_dt = title_dl.find('dt')
    title = title_dt.find('a')['title']
    links = title_dt.find('a')['href']
    link = 'https://comic.naver.com' + links
    print(title, '\t', link) # 제목, 링크

    date = '완결'
    stores = title_dt.find('em',{'class':'ico_store'})
    if stores is not None:
        date += ' '+stores.text
        if stores.text == '스토어':
            date += '(유료)'
    
    star_div = title_dl.find('div',{'class':'rating_type'})
    star_score = star_div.find('strong').text
    print(star_score)
    print('-'*70)
    
    try:
        cur.execute('insert into webtoon (title, date, link, star_score, platform) values (%s, %s, %s, %s, %s)',
                   (title, date, link, star_score, platform))
        conn.commit()
        print('success')
    except:
        conn.rollback()
        print('fail')

라나 	 https://comic.naver.com/webtoon/list?titleId=789425
9.09
----------------------------------------------------------------------
success
보에 	 https://comic.naver.com/webtoon/list?titleId=789426
9.01
----------------------------------------------------------------------
success
소녀행 	 https://comic.naver.com/webtoon/list?titleId=789427
8.89
----------------------------------------------------------------------
success
빨리감기 	 https://comic.naver.com/webtoon/list?titleId=764040
9.91
----------------------------------------------------------------------
success
가슴털 로망스 	 https://comic.naver.com/webtoon/list?titleId=729963
9.92
----------------------------------------------------------------------
success
3cm 헌터 	 https://comic.naver.com/webtoon/list?titleId=733458
9.84
----------------------------------------------------------------------
success
드로잉 레시피 	 https://comic.naver.com/webtoon/list?titleId=750493
9.96
----------------------------------------------------------------------
succ

success
두번째 생일 	 https://comic.naver.com/webtoon/list?titleId=739292
9.43
----------------------------------------------------------------------
success
언플래시레슨 	 https://comic.naver.com/webtoon/list?titleId=762778
6.30
----------------------------------------------------------------------
success
미드나잇 체이서 	 https://comic.naver.com/webtoon/list?titleId=724482
9.61
----------------------------------------------------------------------
success
착하게 살자 시즌1~2 	 https://comic.naver.com/webtoon/list?titleId=776295
9.60
----------------------------------------------------------------------
success
무의식의 숲 	 https://comic.naver.com/webtoon/list?titleId=776296
9.93
----------------------------------------------------------------------
success
너와 나의 눈높이 	 https://comic.naver.com/webtoon/list?titleId=776298
9.85
----------------------------------------------------------------------
success
동트는 로맨스 	 https://comic.naver.com/webtoon/list?titleId=746285
9.98
--------------------------------------------

나의 첫번째 새벽 	 https://comic.naver.com/webtoon/list?titleId=742349
9.94
----------------------------------------------------------------------
success
갓물주 	 https://comic.naver.com/webtoon/list?titleId=731297
9.90
----------------------------------------------------------------------
success
죽어도 로맨스 	 https://comic.naver.com/webtoon/list?titleId=759094
9.79
----------------------------------------------------------------------
success
극야 	 https://comic.naver.com/webtoon/list?titleId=721463
9.96
----------------------------------------------------------------------
success
선녀야 야옹해봐! 	 https://comic.naver.com/webtoon/list?titleId=738547
9.79
----------------------------------------------------------------------
success
커넥트 	 https://comic.naver.com/webtoon/list?titleId=727268
9.58
----------------------------------------------------------------------
success
연하의 맛 	 https://comic.naver.com/webtoon/list?titleId=759093
9.56
--------------------------------------------------------------------

success
악취 	 https://comic.naver.com/webtoon/list?titleId=729946
9.92
----------------------------------------------------------------------
success
금붕어 	 https://comic.naver.com/webtoon/list?titleId=723790
9.96
----------------------------------------------------------------------
success
냄새를 보는 소녀 	 https://comic.naver.com/webtoon/list?titleId=746740
9.97
----------------------------------------------------------------------
success
언럭키 맨션 	 https://comic.naver.com/webtoon/list?titleId=746741
9.98
----------------------------------------------------------------------
success
창공의 타이양사무소 	 https://comic.naver.com/webtoon/list?titleId=746743
9.96
----------------------------------------------------------------------
success
견원지간 로맨스 	 https://comic.naver.com/webtoon/list?titleId=746744
9.65
----------------------------------------------------------------------
success
악마에게 은총을 	 https://comic.naver.com/webtoon/list?titleId=746745
9.76
----------------------------------------------------

success
열대어 	 https://comic.naver.com/webtoon/list?titleId=703629
9.94
----------------------------------------------------------------------
success
펀브로커 	 https://comic.naver.com/webtoon/list?titleId=710765
9.92
----------------------------------------------------------------------
success
완벽하지 않은 키스 	 https://comic.naver.com/webtoon/list?titleId=710757
9.94
----------------------------------------------------------------------
success
나노리스트 	 https://comic.naver.com/webtoon/list?titleId=670150
9.98
----------------------------------------------------------------------
success
반투명인간 	 https://comic.naver.com/webtoon/list?titleId=710762
9.95
----------------------------------------------------------------------
success
나의 짝사랑 고백법 	 https://comic.naver.com/webtoon/list?titleId=709628
9.83
----------------------------------------------------------------------
success
불괴 	 https://comic.naver.com/webtoon/list?titleId=710766
9.40
-----------------------------------------------------------

success
아일랜드 1부 	 https://comic.naver.com/webtoon/list?titleId=678500
9.96
----------------------------------------------------------------------
success
죽은 마법사의 도시 	 https://comic.naver.com/webtoon/list?titleId=409629
9.95
----------------------------------------------------------------------
success
크리퍼스큘 	 https://comic.naver.com/webtoon/list?titleId=297795
9.94
----------------------------------------------------------------------
success
3인칭 	 https://comic.naver.com/webtoon/list?titleId=682265
9.94
----------------------------------------------------------------------
success
첩보의 별 	 https://comic.naver.com/webtoon/list?titleId=675392
9.86
----------------------------------------------------------------------
success
썸남 	 https://comic.naver.com/webtoon/list?titleId=654331
9.97
----------------------------------------------------------------------
success
2016 비명 	 https://comic.naver.com/webtoon/list?titleId=682803
9.14
-----------------------------------------------------------

success
그날의 생존자들 	 https://comic.naver.com/webtoon/list?titleId=557678
9.75
----------------------------------------------------------------------
success
소나기야 	 https://comic.naver.com/webtoon/list?titleId=578106
9.95
----------------------------------------------------------------------
success
2013 루키 단편선 	 https://comic.naver.com/webtoon/list?titleId=597593
9.81
----------------------------------------------------------------------
success
라이징패스트볼 	 https://comic.naver.com/webtoon/list?titleId=414611
9.51
----------------------------------------------------------------------
success
키드갱 시즌2 	 https://comic.naver.com/webtoon/list?titleId=471286
9.94
----------------------------------------------------------------------
success
터치! 메리크리스마스 	 https://comic.naver.com/webtoon/list?titleId=604147
9.78
----------------------------------------------------------------------
success
오렌지 마말레이드 	 https://comic.naver.com/webtoon/list?titleId=293520
9.91
-----------------------------------------

9.84
----------------------------------------------------------------------
success
17살, 그 여름날의 기적 	 https://comic.naver.com/webtoon/list?titleId=76758
9.89
----------------------------------------------------------------------
success
몽타주 	 https://comic.naver.com/webtoon/list?titleId=89098
9.91
----------------------------------------------------------------------
success
탐구생활4-그대와 함께 하이킹 	 https://comic.naver.com/webtoon/list?titleId=142910
9.68
----------------------------------------------------------------------
success
판타스틱 어른백서 	 https://comic.naver.com/webtoon/list?titleId=78805
9.03
----------------------------------------------------------------------
success
겨울동화 	 https://comic.naver.com/webtoon/list?titleId=113121
9.89
----------------------------------------------------------------------
success
악연 	 https://comic.naver.com/webtoon/list?titleId=94760
9.90
----------------------------------------------------------------------
success
색으로 말하다 	 https://comic.naver.com/webt

In [ ]:
# try:
#     cur.execute('insert into webtoon (title, date, link, star_score, platform) values (%s, %s, %s, %s, %s)',
#                (title, date, link, star_score, platform))
#     conn.commit()
#     print('success')
# except:
#     conn.rollback()
#     print('fail')

success


![image.png](attachment:image.png)

## 네이버 웹툰 (작가 테이블)

- 0. import 및 DB 연결

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import datetime as dt
from selenium.webdriver.common.by import By
import psycopg2

In [ ]:
try:
    conn = psycopg2.connect(host="localhost",database="220205", user="postgres", password="admin",port=5432)
    print('===접속 성공===')
    cur = conn.cursor()
except psycopg2.DatabaseError as db_err:
    print('접속 오류 !!')
    print(db_err)

===접속 성공===


### 작가 테이블 생성, 정보 크롤링  

#### 작가 테이블 생성

In [ ]:
# 작가 테이블 생성

cur = conn.cursor()
try:
    cur.execute('CREATE TABLE if not exists writer (id serial, writer varchar(100), found boolean);')
    conn.commit()
    print('success')
except:
    print('fail')

success


![image-2.png](attachment:image-2.png)

* 작가 검색시 생기는 문제점
  1. 동명이인
  2. 인물사전에 작가정보가 없음 (본인이 삭제시 네이버가 삭제해주는 경우 or 아직 등록되지 않은 경우)
  3. 인물사전에 들어 있는 정보들이 다 다름
  4. 정보에 링크가 있기도 하고 없기도 함

* 문제점 해결 방안
  1. 작가 이름 앞에 '만화가 ' 를 붙여서 검색
  2. found 컬럼을 만들어서 정보가 없으면 '찾지 못했다'라는 의미로 'false' 값을 줌
  3. 각 컬럼별로 정보를 저장하고, 컬럼이 존재하지 않아서 에러가 발생한 경우 자동으로 컬럼을 만들어서 다시 저장하는 함수를 만듦
  4. 링크가 있을 경우, 3.에서 만든 함수를 활용해 컬럼이름에 ~ 'link_1' 이라는 이름으로 새로 생성하여 저장함

![image.png](attachment:image.png) ![image-3.png](attachment:image-3.png)
![image-4.png](attachment:image-4.png) ![image-5.png](attachment:image-5.png)


#### 컬럼이 없을 때 고쳐서 넣어주는 코드 함수화

In [ ]:
# 컬럼이 없을 경우 컬럼을 만든 뒤 다시 집어넣어주는 함수 정의

def save_data_column(column_name, column_data, writer_name):
    cur = conn.cursor()
    for i in range(2):
        try:
            cur.execute("update writer set {0} = '{1}' where writer = '{2}';".format(column_name,column_data,writer_name))
            conn.commit()
#             print('success')
            break
        except psycopg2.ProgrammingError as pg_err:
#             print('save_data_column ProgrammingError')
#             print(pg_err)
            conn.rollback()
            cur.execute("alter table writer add column {0} varchar(500);".format(column_name))
            conn.commit()
#             print(' 재 시 도 ')

#         except psycopg2.DatabaseError as db_err:
#             print('save_data_column DatabaseError')
# #             print(db_err)
#             conn.rollback()
    
        except:
            print('save_data_column Fail')
            conn.rollback()
    cur.close()

- 만화가 모랑지/SIU 로 TEST (T 모랑지 F SIU)

In [ ]:
writer_name = '모랑지' # 인물 사전을 정상적으로 찾는 경우
search_and_update(writer_name)

print('-'*70)

writer_name = 'SIU' # 인물 사전에 없는 경우
search_and_update(writer_name)

search_and_update End
----------------------------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End


#### 작가 이름을 입력 받아 → 네이버 검색정보 크롤링 → DB 저장    까지 해주는 함수 생성

In [ ]:
def search_and_update(writer_name):
    words = '만화가 '+writer_name
    word = words.replace(' ','+')
    
    url ='https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query='+word
    response=requests.get(url)
    response.raise_for_status()
    
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    name_find_div = soup.find('div',{'class':'title_area type_keep _title_area'})
    try:
        name_find = name_find_div.find('span',{'class':'area_text_title'})
        search_name = name_find.text # 작가 이름 (검색된 이름)

        profile_box_div = soup.find('div',{'class':'cm_info_box'}) # 프로필 박스
        profile_info_txt = profile_box_div.find('dl',{'class':'info txt_3'})
        profile_all = profile_info_txt.find_all('div',{'class':'info_group'})
        
        cur = conn.cursor()
        try:
            cur.execute("insert into writer (writer, found) values ('{0}', 'T');".format(writer_name))
            conn.commit()
#             print('success')
        except psycopg2.DatabaseError as db_err:
            print('search_and_update DatabaseError')
            print(db_err)
            conn.rollback()
        except:
            print('search_and_update --------Insert Fail---------')
            conn.rollback()

        for profile in profile_all: # 프로필 정보에 대한 것들 중에 1개의 정보씩
            what_profile = profile.find('dt').text
            profile_data = profile.find('dd') #.text
            profile_data_fix = profile_data.text.replace("'","''")
            x = profile_data.find_all('a')
#             print(profile_data_fix)

            save_data_column(what_profile, profile_data_fix, writer_name) # href 제외한 text만 업데이트


            for i in range(len(x)):
                try:
                    exec("what_profile_{0} = what_profile +'link_'+'{1}'".format(i+1,i+1))
                    exec("href_{0} = x[{1}]['href']".format(i+1, i))
                    exec("save_data_column(what_profile_{0}, href_{1}, writer_name)".format(+1, i+1))
                except TypeError:
                    print('search_and_update TypeError')
                except IndexError:
                    print('search_and_update Index Error')

#             try:
#                 profile_href = profile.find('dd')['href']
#                 print(what_profile,'\t/',profile_data,'\t/',profile_href)
#             except:
#                 print(what_profile,'\t/',profile_data)


            
            
            
    except AttributeError as at_err:
        print("정보가 없습니다. (Found 컬럼 False)")
        cur = conn.cursor()
        try:
            cur.execute("insert into writer (writer, found) values ('{0}', 'F');".format(writer_name))
            conn.commit()
#             print('success')
        except psycopg2.DatabaseError as db_err:
            print('search_and_update (False) DatabaseError')
#             print(db_err)
            conn.rollback()
    
        except:
            print('search_and_update (False) Fail')
            conn.rollback()
    print('search_and_update End')

### 웹툰 내에 있는 작가 이름 리스트 생성

- 웹툰 테이블 안에 있는 작가 정보를 모아 아래와 같은 과정을 거친 후 중복이 없이 작가 이름이 있는 list를 만듦
    - 작가가 다수인 경우 ' / ' 로 분리
    - 중복된 이름 정리

In [ ]:
try:
    cur.execute("select writer from webtoon")
    writer_fetchall=cur.fetchall()
    conn.commit()
    print('success')
except:
    print('fail')
    conn.rollback()

writer_list = []
for writer in writer_fetchall:
    writer_names = writer[0].split(' / ')
    for writer_name in writer_names:
        writer_list.append(writer_name)

print('중복 전 : '+str(len(writer_list)) + '개 있음')
writer_list = list(set(writer_list))
print('중복 정리 후 : '+str(len(writer_list)) + '개 있음')
print(writer_list[:5]) # 5개만 보기

success
중복 전 : 2198개 있음
중복 정리 후 : 1465개 있음
['박화랑', '-2℃', '요나', '인미', 'JMS']


* 작가 이름 리스트로 함수 실행 (진행 정보 볼 수 있게끔)

In [ ]:
start = time.time()
for i, writer_name in enumerate(writer_list):
    search_and_update(writer_name)
    print(writer_name +'\t'+str(i+1)+'/'+str(len(writer_list)))
    print('-'*50)
end = time.time()
print(str(end-start) + '초')

search_and_update End
박화랑	1/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
-2℃	2/1465
--------------------------------------------------
search_and_update End
요나	3/1465
--------------------------------------------------
search_and_update End
인미	4/1465
--------------------------------------------------
search_and_update End
JMS	5/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
건치	6/1465
--------------------------------------------------
search_and_update End
양수	7/1465
--------------------------------------------------
search_and_update End
주다현	8/1465
--------------------------------------------------
search_and_update End
문아	9/1465
--------------------------------------------------
search_and_update End
텐	10/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
이한빈	11/1465
--------------------------------------------------


search_and_update End
카라쿨	90/1465
--------------------------------------------------
search_and_update End
김동준	91/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
비로	92/1465
--------------------------------------------------
search_and_update End
여슬기	93/1465
--------------------------------------------------
search_and_update End
김순돌	94/1465
--------------------------------------------------
search_and_update End
전마두	95/1465
--------------------------------------------------
search_and_update End
두지	96/1465
--------------------------------------------------
search_and_update End
나윤희	97/1465
--------------------------------------------------
search_and_update End
박새	98/1465
--------------------------------------------------
search_and_update End
고다	99/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
박성진	100/1465
--------------------------------------------------
정보가 없습니다. (Fou

search_and_update End
유비	180/1465
--------------------------------------------------
search_and_update End
마주	181/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
불사	182/1465
--------------------------------------------------
search_and_update End
제뉴	183/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
이주운	184/1465
--------------------------------------------------
search_and_update End
돌배	185/1465
--------------------------------------------------
search_and_update End
서재일	186/1465
--------------------------------------------------
search_and_update End
김진석	187/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
견지	188/1465
--------------------------------------------------
search_and_update End
강호진	189/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
꼬마비,앙마비	190/1465

정보가 없습니다. (Found 컬럼 False)
search_and_update End
울	267/1465
--------------------------------------------------
search_and_update End
닺	268/1465
--------------------------------------------------
search_and_update End
유성연	269/1465
--------------------------------------------------
search_and_update End
곤세	270/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
허니맨	271/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
변기현	272/1465
--------------------------------------------------
search_and_update End
고긱	273/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
비가	274/1465
--------------------------------------------------
search_and_update End
켄타	275/1465
--------------------------------------------------
search_and_update End
수연	276/1465
--------------------------------------------------
search_and_update End
여은	277/1465
-------

정보가 없습니다. (Found 컬럼 False)
search_and_update End
진자	356/1465
--------------------------------------------------
search_and_update End
유현	357/1465
--------------------------------------------------
search_and_update End
조양	358/1465
--------------------------------------------------
search_and_update End
영재영	359/1465
--------------------------------------------------
search_and_update End
이솔	360/1465
--------------------------------------------------
search_and_update End
배드이리	361/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
기매	362/1465
--------------------------------------------------
search_and_update End
양영순	363/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
문설희	364/1465
--------------------------------------------------
search_and_update End
여백	365/1465
--------------------------------------------------
search_and_update End
김다찌	366/1465
-----------------------------

...

정보가 없습니다. (Found 컬럼 False)
search_and_update End
서혜은	1057/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
ELDO	1058/1465
--------------------------------------------------
search_and_update End
코어	1059/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
여우	1060/1465
--------------------------------------------------
search_and_update End
석한	1061/1465
--------------------------------------------------
search_and_update End
박태준	1062/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
Big Hit Ent.	1063/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
메슬로우	1064/1465
--------------------------------------------------
search_and_update End
김지수	1065/1465
--------------------------------------------------
search_and_update End
재희	1066/1465
---------------------------------------

search_and_update End
힐링달	1144/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
달콤J,박성실	1145/1465
--------------------------------------------------
search_and_update End
JINHA	1146/1465
--------------------------------------------------
search_and_update End
카레곰	1147/1465
--------------------------------------------------
search_and_update End
세혼	1148/1465
--------------------------------------------------
search_and_update End
가천가	1149/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
이상신	1150/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
태소영	1151/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
0환이	1152/1465
--------------------------------------------------
search_and_update End
연상호	1153/1465
--------------------------------------------------
정보가 없습니다. (Found 

정보가 없습니다. (Found 컬럼 False)
search_and_update End
성불예정,홍실	1230/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
유진성	1231/1465
--------------------------------------------------
search_and_update End
김용키	1232/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
아인	1233/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
뻥	1234/1465
--------------------------------------------------
search_and_update End
랑또	1235/1465
--------------------------------------------------
search_and_update End
예타쿠	1236/1465
--------------------------------------------------
search_and_update End
예랑	1237/1465
--------------------------------------------------
search_and_update End
강냉이	1238/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
이순기	1239/1465
-----------------------------------------------

정보가 없습니다. (Found 컬럼 False)
search_and_update End
도미	1318/1465
--------------------------------------------------
search_and_update End
김용진	1319/1465
--------------------------------------------------
search_and_update End
김학영	1320/1465
--------------------------------------------------
search_and_update End
정샛별	1321/1465
--------------------------------------------------
search_and_update End
자유	1322/1465
--------------------------------------------------
search_and_update End
엽	1323/1465
--------------------------------------------------
search_and_update End
구구	1324/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
해밀k	1325/1465
--------------------------------------------------
search_and_update End
범우	1326/1465
--------------------------------------------------
search_and_update End
즐바센	1327/1465
--------------------------------------------------
search_and_update End
차용운	1328/1465
----------------------------------------------

search_and_update End
반지은	1405/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
비버	1406/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
김작가	1407/1465
--------------------------------------------------
search_and_update End
국중록	1408/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
이지훈	1409/1465
--------------------------------------------------
search_and_update End
루시드	1410/1465
--------------------------------------------------
search_and_update End
아흐레달	1411/1465
--------------------------------------------------
search_and_update End
이윤희	1412/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
엄재경	1413/1465
--------------------------------------------------
정보가 없습니다. (Found 컬럼 False)
search_and_update End
송채윤	1414/1465
----------------------------------------------

* 최종 생성된 테이블의 모양

![image.png](attachment:image.png)

## 네이버 웹툰 서브 테이블 (웹툰 별 매화 정보를 담고 있는 테이블)
    * "웹툰 이름" 테이블에 가까움 ( ex-신의탑 테이블, 여신강림 테이블 )
        * 각 웹툰의 매화에 대한 정보들을 담을 예정 ( ex-신의탑 1화 별점, 조회수, 부제목, 연재일, 해당화 링크 등 )
        * 생성해야할 테이블이 너무 많아 일부만 선정하여 생성

![image.png](attachment:image.png)

- Import

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import datetime as dt
from selenium.webdriver.common.by import By
import psycopg2

In [ ]:
url = 'https://comic.naver.com/webtoon/weekday'

response = requests.get(url)
response.raise_for_status()
html=response.content
soup=BeautifulSoup(html, 'html.parser')

In [ ]:
try:
    conn = psycopg2.connect(host="localhost",database="Postgres", user="postgres", password="admin",port=5432)
    print('===접속 성공===')
    cur = conn.cursor()
except psycopg2.DatabaseError as db_err:
    print('접속 오류 !!')
    print(db_err)

    cur = conn.cursor()
try:
    cur.execute('CREATE TABLE if not exists webtoon (id serial, date varchar(50), title varchar(200), link varchar(500), writer varchar(200), platform varchar(200));')
    conn.commit()
    print('success')
except:
    print('fail')

===접속 성공===
success


### 작품 추출
* 예시로 '신의 탑' 이라는 웹툰 만 크롤링 하였음

In [ ]:

url_list = []
url_list.append('https://comic.naver.com/webtoon/list?titleId=183559') # Webtoon 테이블에 있는 Link 들을 추출해서 사용 가능

for url in url_list:
    response = requests.get(url)
    response.raise_for_status()
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    
    divs=soup.find('div', {'id' : 'content'})
    title_trs=divs.find_all('td', {'class' : 'title'})
    wkstar_divs = divs.find_all('div', {'class':'rating_type'}) 
    upload_trs = divs.find_all('tr')
    title_list=[]
    link_list=[]

    
    for tr in title_trs:
        title = tr.find('a').text                        # 제목
        links = tr.find('a')['href']                     # 링크
        link = 'https://comic.naver.com' + links         # 링크 정리
        print(title, '\t', link)                         # 제목과 링크 출력
        print('-'*70)
        
        
        
#         회 별 평점 크롤링
    star_list=[]
    for tr in wkstar_divs:
        wkstar_score = tr.find('strong').text
        print("회별 평점 : ", wkstar_score)
        print('-'*70)
    
    
#         등록일 정보 크롤링
    upload_list=[]
    cnt =0
    for tr in upload_trs:
        cnt += 1
        if cnt >= 4:
            tds = tr.find_all('td')
            print("등록일 : ", tds[3].text)
            print('-'*70)
            
        upload = tr.find('td', {'class' : 'num'}).text
        print(upload)
        print('등록일 : ' )

#         try:
#             cur.execute('update webtoon set wkstar_score = %s where link = %s',
#                        (wkstar_score, link))
#             conn.commit()
#             print('success')
#         except:
#             conn.rollback()
#             print('fail')

3부 107화 	 https://comic.naver.com/webtoon/detail?titleId=183559&no=526&weekday=mon
----------------------------------------------------------------------
3부 106화 	 https://comic.naver.com/webtoon/detail?titleId=183559&no=525&weekday=mon
----------------------------------------------------------------------
3부 105화 	 https://comic.naver.com/webtoon/detail?titleId=183559&no=524&weekday=mon
----------------------------------------------------------------------
3부 104화 	 https://comic.naver.com/webtoon/detail?titleId=183559&no=523&weekday=mon
----------------------------------------------------------------------
3부 103화 	 https://comic.naver.com/webtoon/detail?titleId=183559&no=522&weekday=mon
----------------------------------------------------------------------
3부 102화 	 https://comic.naver.com/webtoon/detail?titleId=183559&no=521&weekday=mon
----------------------------------------------------------------------
3부 101화 	 https://comic.naver.com/webtoon/detail?titleId=183559&no=520&weekd

- 테이블 생성

## 네이버 웹툰 (댓글 테이블)

* 해당 화에 들어가서, 댓글 부분을 크롤링한 결과, 좋아요 모듈과 비슷한 이슈가 발생하였음 (댓글 모듈 존재)  
  requests 모듈 (정적 크롤링)으로는 댓글 모듈을 실행시키지 않고 크롤링하여 댓글을 읽을 수 없었음  
  그래서, 셀레니움 모듈(동적 크롤링)로 크롤링을 진행함

* import

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import datetime as dt
from selenium.webdriver.common.by import By
import psycopg2

In [ ]:
base_url = 'https://comic.naver.com/webtoon/weekday.nhn'
driver = webdriver.Chrome()
def drive(url): 
    driver = webdriver.Chrome('./chromedriver') #driver 객체 불러옴 
    driver.implicitly_wait(3) # 3초 후에 작동하도록 
    driver.get(url) #url에 접속 
    html = driver.page_source #현재 driver에 나타난 창의 page_source(html) 가져오기 
    soup = BeautifulSoup(html, 'html.parser') #html 파싱(parsing)을 위해 BeautifulSoup에 넘겨주기 
    return driver, soup
driver, soup = drive(base_url)
driver.close()

#가져온 데이터 파싱, id, 요일, 이름
title = soup.select('.title')
t_IDs=list(map(lambda x: x.get('href').split('titleId=')[1].split('&')[0], title))

In [ ]:
t_IDs

['783054',
 '758037',
 '183559',
 '787411',
 '783052',
 '654774',
 '728750',
 '733074',
 '774863',
 '739115',
 '768536',
 '759940',
 '703849',
 '752414',
 '786979',
 '769663',
 '21815',
 '730174',
 '772725',
 '774866',
 '741891',
 '597478',
 '758671',
 '758666',
 '783050',
 '687915',
 '774870',
 '738809',
 '788170',
 '753858',
 '785253',
 '774703',
 '783535',
 '770656',
 '789591',
 '780414',
 '786910',
 '780881',
 '764623',
 '744320',
 '768472',
 '778329',
 '783525',
 '774040',
 '767919',
 '772734',
 '785250',
 '752350',
 '752413',
 '752530',
 '772243',
 '784107',
 '778748',
 '740132',
 '774302',
 '765776',
 '761463',
 '767979',
 '764041',
 '776256',
 '764480',
 '770590',
 '769801',
 '734918',
 '744375',
 '785259',
 '745654',
 '756790',
 '754872',
 '785749',
 '758667',
 '783054',
 '783053',
 '703846',
 '787411',
 '773522',
 '743139',
 '784248',
 '730656',
 '738487',
 '738174',
 '683496',
 '759925',
 '772401',
 '773459',
 '721433',
 '711422',
 '702608',
 '719508',
 '783877',
 '783057',


In [ ]:
url = 'https://comic.naver.com/webtoon/weekday'

response = requests.get(url)
response.raise_for_status()
html=response.content
soup=BeautifulSoup(html, 'html.parser')

driver = webdriver.Chrome('chromedriver') 
driver.get('https://comic.naver.com/webtoon/detail?titleId={}'.format(t_IDs[0]))
driver.maximize_window()

In [ ]:
try:
    conn = psycopg2.connect(host="localhost",database="220205", user="postgres", password="admin",port=5432)
    print('===접속 성공===')
    cur = conn.cursor()
except psycopg2.DatabaseError as db_err:
    print('접속 오류 !!')
    print(db_err)



===접속 성공===


In [ ]:
int(len(t_IDs)/10)

51

In [ ]:
import pandas as pd
import numpy as np

import time

### 댓글을 크롤링 해주는 함수 지정

In [ ]:
# target = int(len(t_IDs)/10)

def collector(driver, data, k):
    driver = webdriver.Chrome("chromedriver.exe")
    for i in range(14) :
    #range값 target 변수 넣어도 된다. 51개
        
    #url = 'https://comic.naver.com/webtoon/detail.nhn?titleId=641253&no='+str(k)
        url = 'https://comic.naver.com/webtoon/detail.nhn?titleId=' + t_IDs[i] + '&no=' + str(k) # 예시 웹툰 1종 선정
              # 앞부분을 기존 테이블 링크로 대체 가능
            
        driver.get(url)
        time.sleep(1)
    
    # 댓글창에 해당하는 프레임 선택
        driver.switch_to.frame('commentIframe')
    
    # 회차,이름,날짜,아이디,댓글,좋아요/싫어요 수 전체 수집
        num = k
        names = driver.find_elements_by_css_selector('.u_cbox_nick')
        dates = driver.find_elements_by_css_selector('.u_cbox_date')
        nicks = driver.find_elements_by_css_selector('.u_cbox_id')
        reviews = driver.find_elements_by_css_selector('.u_cbox_text_wrap')
        likes = driver.find_elements_by_css_selector('.u_cbox_recomm_set')

        
    
    # 총 n개의 댓글이 수집되므로 해당 댓글 수만큼 for문
        for i in range(len(names)):
        # for문을 돌며, 댓글별 회차,날짜,이름,아이디,댓글,좋아요/싫어요를 저장
            temp = []
            temp.append(t_IDs[i])
            temp.append(num)
            temp.append(dates[i].text)
            temp.append(names[i].text)
            temp.append(nicks[i].text)
            temp.append(reviews[i].text)
            temp.append(likes[i].text)

            temp = pd.DataFrame(data=[temp], columns=data.columns)
            data = pd.concat([data,temp])

    # 열어두었던 driver를 닫기
    driver.close()

    print(str(num) + '화 베댓 수집 완료')

    return data

### 크롤링

In [ ]:
k=199
data=pd.DataFrame(index=range(0,0), columns=['tableid','num','dates','names','nicks','reviews','likes'])
data1 = collector(driver, data, k)
print(data1)


199화 베댓 수집 완료
   tableid  num             dates       names       nicks  \
0   783054  199  2021-12-28 22:56         밝기맨  (give****)   
0   758037  199  2021-12-28 23:10        개밥그릇  (hgh2****)   
0   183559  199  2021-12-28 23:20  NoSignal00  (mari****)   
0   787411  199  2021-12-28 23:03         SSJ  (cc91****)   
0   783052  199  2021-12-28 23:22     Djdndis  (0214****)   
..     ...  ...               ...         ...         ...   
0   783054  199  2022-01-09 23:03         유세진  (pp46****)   
0   758037  199  2022-01-09 23:31         rhk  (rhkr****)   
0   183559  199  2022-01-09 23:55        애요옹이  (phon****)   
0   787411  199  2022-01-09 23:38        시구장구  (jw08****)   
0   783052  199  2022-01-10 01:15         강성운  (sap0****)   

                                              reviews               likes  
0                              BEST락커때려눕힌다음엔 발라드황제나옴?  좋아요\n3635\n싫어요\n13  
0            BEST나도하 특: 소개할때는 최종보스급으로 나오다 나중엔 전투력측정기됨  좋아요\n2639\n싫어요\n13  
0   BEST1.튜토리얼: 랩배틀 동아리 회

In [ ]:
data1

,tableid,num,dates,names,nicks,reviews,likes
0,783054,199,2021-12-28 22:56,밝기맨,(give****),BEST락커때려눕힌다음엔 발라드황제나옴?,좋아요\n3635\n싫어요\n13
0,758037,199,2021-12-28 23:10,개밥그릇,(hgh2****),BEST나도하 특: 소개할때는 최종보스급으로 나오다 나중엔 전투력측정기됨,좋아요\n2639\n싫어요\n13
0,183559,199,2021-12-28 23:20,NoSignal00,(mari****),BEST1.튜토리얼: 랩배틀 동아리 회장(래퍼)2.무학고편 흑막&천산고편 보스: '...,좋아요\n1401\n싫어요\n23
0,787411,199,2021-12-28 23:03,SSJ,(cc91****),BEST놀랍게도 배틀만화의 정석을 밟고 있음. 반박시 니 말이 맞음.비범한 강함을 ...,좋아요\n1322\n싫어요\n12
0,783052,199,2021-12-28 23:22,Djdndis,(0214****),BEST내가볼땐 얘네 걍 쇼미나가서 우승자 줘패고 트로피 강탈하고 엔딩날듯,좋아요\n1110\n싫어요\n8
...,...,...,...,...,...,...,...
0,783054,199,2022-01-09 23:03,유세진,(pp46****),BEST그림체 ㅈㄴ신경쓴게 보인다,좋아요\n3141\n싫어요\n23
0,758037,199,2022-01-09 23:31,rhk,(rhkr****),BEST지수 중간에 질투하는것 같은데 언급이 없네요ㅋㅋ기린 머리 쓰다듬어 줄때 지수...,좋아요\n1979\n싫어요\n4
0,183559,199,2022-01-09 23:55,애요옹이,(phon****),BEST작화 좋아진게 이렇게 눈에 띌 정도는 잘 없는데 정말 신경 많이 쓰셨나봐요그...,좋아요\n1551\n싫어요\n16
0,787411,199,2022-01-09 23:38,시구장구,(jw08****),BEST몽상시리즈 스포아닌 스포하면 개쩔어요,좋아요\n918\n싫어요\n23


In [ ]:
tmp = data1.copy()
tmp

,tableid,num,dates,names,nicks,reviews,likes
0,783054,199,2021-12-28 22:56,밝기맨,(give****),BEST락커때려눕힌다음엔 발라드황제나옴?,좋아요\n3635\n싫어요\n13
0,758037,199,2021-12-28 23:10,개밥그릇,(hgh2****),BEST나도하 특: 소개할때는 최종보스급으로 나오다 나중엔 전투력측정기됨,좋아요\n2639\n싫어요\n13
0,183559,199,2021-12-28 23:20,NoSignal00,(mari****),BEST1.튜토리얼: 랩배틀 동아리 회장(래퍼)2.무학고편 흑막&천산고편 보스: '...,좋아요\n1401\n싫어요\n23
0,787411,199,2021-12-28 23:03,SSJ,(cc91****),BEST놀랍게도 배틀만화의 정석을 밟고 있음. 반박시 니 말이 맞음.비범한 강함을 ...,좋아요\n1322\n싫어요\n12
0,783052,199,2021-12-28 23:22,Djdndis,(0214****),BEST내가볼땐 얘네 걍 쇼미나가서 우승자 줘패고 트로피 강탈하고 엔딩날듯,좋아요\n1110\n싫어요\n8
...,...,...,...,...,...,...,...
0,783054,199,2022-01-09 23:03,유세진,(pp46****),BEST그림체 ㅈㄴ신경쓴게 보인다,좋아요\n3141\n싫어요\n23
0,758037,199,2022-01-09 23:31,rhk,(rhkr****),BEST지수 중간에 질투하는것 같은데 언급이 없네요ㅋㅋ기린 머리 쓰다듬어 줄때 지수...,좋아요\n1979\n싫어요\n4
0,183559,199,2022-01-09 23:55,애요옹이,(phon****),BEST작화 좋아진게 이렇게 눈에 띌 정도는 잘 없는데 정말 신경 많이 쓰셨나봐요그...,좋아요\n1551\n싫어요\n16
0,787411,199,2022-01-09 23:38,시구장구,(jw08****),BEST몽상시리즈 스포아닌 스포하면 개쩔어요,좋아요\n918\n싫어요\n23


* 좋아요 싫어요 분리

In [ ]:
tmp['reviews'] = data1['reviews'].str.replace('BEST',"")
#tmp['reviews'] = data1['reviews'].str.replace('BEST',"")
#data1['likes'] = data1['likes'].str.replace('\n'," ")
tmp['좋아요'] = tmp['likes'].str.split('\n').str[1]
tmp['싫어요'] = tmp['likes'].str.split('\n').str[3]

In [ ]:
tmp

,tableid,num,dates,names,nicks,reviews,likes,좋아요,싫어요
0,783054,199,2021-12-28 22:56,밝기맨,(give****),락커때려눕힌다음엔 발라드황제나옴?,좋아요\n3635\n싫어요\n13,3635,13
0,758037,199,2021-12-28 23:10,개밥그릇,(hgh2****),나도하 특: 소개할때는 최종보스급으로 나오다 나중엔 전투력측정기됨,좋아요\n2639\n싫어요\n13,2639,13
0,183559,199,2021-12-28 23:20,NoSignal00,(mari****),1.튜토리얼: 랩배틀 동아리 회장(래퍼)2.무학고편 흑막&천산고편 보스: '죽음에서...,좋아요\n1401\n싫어요\n23,1401,23
0,787411,199,2021-12-28 23:03,SSJ,(cc91****),놀랍게도 배틀만화의 정석을 밟고 있음. 반박시 니 말이 맞음.비범한 강함을 지닌 주...,좋아요\n1322\n싫어요\n12,1322,12
0,783052,199,2021-12-28 23:22,Djdndis,(0214****),내가볼땐 얘네 걍 쇼미나가서 우승자 줘패고 트로피 강탈하고 엔딩날듯,좋아요\n1110\n싫어요\n8,1110,8
...,...,...,...,...,...,...,...,...,...
0,783054,199,2022-01-09 23:03,유세진,(pp46****),그림체 ㅈㄴ신경쓴게 보인다,좋아요\n3141\n싫어요\n23,3141,23
0,758037,199,2022-01-09 23:31,rhk,(rhkr****),지수 중간에 질투하는것 같은데 언급이 없네요ㅋㅋ기린 머리 쓰다듬어 줄때 지수 밑에 ...,좋아요\n1979\n싫어요\n4,1979,4
0,183559,199,2022-01-09 23:55,애요옹이,(phon****),작화 좋아진게 이렇게 눈에 띌 정도는 잘 없는데 정말 신경 많이 쓰셨나봐요그림을 보...,좋아요\n1551\n싫어요\n16,1551,16
0,787411,199,2022-01-09 23:38,시구장구,(jw08****),몽상시리즈 스포아닌 스포하면 개쩔어요,좋아요\n918\n싫어요\n23,918,23


In [ ]:
tmp = tmp.drop('likes', axis=1) 



In [ ]:
data_reviews = tmp

In [ ]:
data_reviews

,tableid,num,dates,names,nicks,reviews,좋아요,싫어요
0,783054,199,2021-12-28 22:56,밝기맨,(give****),락커때려눕힌다음엔 발라드황제나옴?,3635,13
0,758037,199,2021-12-28 23:10,개밥그릇,(hgh2****),나도하 특: 소개할때는 최종보스급으로 나오다 나중엔 전투력측정기됨,2639,13
0,183559,199,2021-12-28 23:20,NoSignal00,(mari****),1.튜토리얼: 랩배틀 동아리 회장(래퍼)2.무학고편 흑막&천산고편 보스: '죽음에서...,1401,23
0,787411,199,2021-12-28 23:03,SSJ,(cc91****),놀랍게도 배틀만화의 정석을 밟고 있음. 반박시 니 말이 맞음.비범한 강함을 지닌 주...,1322,12
0,783052,199,2021-12-28 23:22,Djdndis,(0214****),내가볼땐 얘네 걍 쇼미나가서 우승자 줘패고 트로피 강탈하고 엔딩날듯,1110,8
...,...,...,...,...,...,...,...,...
0,783054,199,2022-01-09 23:03,유세진,(pp46****),그림체 ㅈㄴ신경쓴게 보인다,3141,23
0,758037,199,2022-01-09 23:31,rhk,(rhkr****),지수 중간에 질투하는것 같은데 언급이 없네요ㅋㅋ기린 머리 쓰다듬어 줄때 지수 밑에 ...,1979,4
0,183559,199,2022-01-09 23:55,애요옹이,(phon****),작화 좋아진게 이렇게 눈에 띌 정도는 잘 없는데 정말 신경 많이 쓰셨나봐요그림을 보...,1551,16
0,787411,199,2022-01-09 23:38,시구장구,(jw08****),몽상시리즈 스포아닌 스포하면 개쩔어요,918,23


* 분석을 위한 CSV 저장

In [ ]:
data_reviews.to_csv("comment_webtoon6.csv", index=False, encoding="utf-8-sig")

In [ ]:
data_reviews.isnull()

,tableid,num,dates,names,nicks,reviews,좋아요,싫어요
0,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
0,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False


### ML 감정분석

- import 및 ML 모듈  
    - 줄별로 split한 데이터를 반복문을 통해 띄어쓰기를 구분자로 split한 것을 리스트로 반환하였고  
      헤더는 제외하여 데이터 파일을 읽었습니다.  
      토크나이징으로 하나의 문장을 토큰화 한 뒤, 텍스트와 품사 태깅을 슬래쉬를 구분자로 묶어줬습니다.  
      리스트 안에 한문장에 대해서 위에서 만든 토크나이징 함수를 통해서 토큰화 텍스트, 긍정 부정여부를 리스트이 각 문장별로 요소로 넣었습니다.  

- IDE : Jupyter Notebook (Anaconda)
- OS : Windows 10 x64
- CPU : Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz   1.80 GHz
- RAM : 16.0 GB
- train data : 150,000개
- test data : 50,000개
- 모델 학습 소요시간 : 66분
- 최종결과 도출 소요시간 : 1시55분부터 시작 2시 17분 끝. 22분 소요.
- 데이터 양 (댓글 수) : 592개

In [ ]:
import json
import os
import nltk
from konlpy.tag import Okt



def open_txt(filename):
    with open(filename, 'r',encoding = 'utf-8') as f:  
#        data= [line.split('\t') for line in f.read()]
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

train_data = open_txt('train_data.txt')
test_data = open_txt('test_data.txt')

okt = Okt()
okt.pos('동메달 딴거 ㅈㄴ좋으면 개추')
# 하나의 문장을 토큰화 한 후 텍스트와 품사태깅을 / 구분자로 묶어준다.
def tokenizing(docs):
    return ['/'.join(t) for t in okt.pos(docs, norm=True, stem=True)]

train_list = []  
test_list = []  
for i in train_data:
    try:
        train_value = [tokenizing(i[1]), i[2]]
        train_list.append(train_value)
    except:
        pass

for i in test_data:
    try:
        test_value = [tokenizing(i[1]), i[2]]
        test_list.append(test_value)
    except:
        pass

tokens = [t for d in train_list for t in d[0]]
text = nltk.Text(tokens,name='NMSC')
text.vocab().most_common(10) #vocab().most_common(10) - 텍스트 빈도 상위 10개 보여주기
selected_words = [f[0] for f in text.vocab().most_common(10000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_list]

# 1번째리뷰를 상위 10000개와 각각 매칭하여 각 10000개의 단어가 해당 문장에 얼마나 포함되는지를 확인
test_x = [term_frequency(d) for d, _ in test_list]

train_y = [c for _, c in train_list]
test_y = [c for _, c in test_list]

import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')


from keras import losses
from keras import metrics
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,))) 
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# 모델 생성
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

# 예측 결과
results

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
150000/150000 [==============================] - 11s 71us/step - loss: 0.3867 - binary_accuracy: 0.8351
Epoch 2/10
150000/150000 [==============================] - 139s 929us/step - loss: 0.3156 - binary_accuracy: 0.8658
Epoch 3/10
150000/150000 [==============================] - 10s 70us/step - loss: 0.2926 - binary_accuracy: 0.8781
Epoch 4/10
150000/150000 [==============================] - 10s 69us/step - loss: 0.2734 - binary_accuracy: 0.8893
Epoch 5/10
150000/150000 [==============================] - 10s 66us/step - loss: 0.2549 - binary_accuracy: 0.8980
Epoch 6/10
150000/150000 [==============================] - 9s 61us/step - loss: 0.2358 - binary_accuracy: 0.9072: 0s - loss: 0.2358 - binary_accuracy: 0.9
Epoch 7/10
150000/150000 [==============================] - 9s 60us/step - loss: 0.2168 - b

[0.4050483472967148, 0.8519799709320068]

In [ ]:
def input_text(text):
    token = tokenizing(text)
    tf =term_frequency(token)

    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)

    score = float(model.predict(data)) #새로운 데이터를 받으면 결과 예측
    if(score > 0.5):
        print("{} : 긍정 [{:.2f}%] \n".format(text, score * 100))
    else:
        print("{} : 부정 [{:.2f}%] \n".format(text, (1 - score) * 100))
    

In [ ]:
import pandas as pd

* 댓글 CSV 파일을 DF으로 읽기

In [ ]:
df1 = pd.read_csv('comment_webtoon6.csv')

In [ ]:
df1

,tableid,num,dates,names,nicks,reviews,좋아요,싫어요
0,783054,199,2021-12-28 22:56,밝기맨,(give****),락커때려눕힌다음엔 발라드황제나옴?,3635,13
1,758037,199,2021-12-28 23:10,개밥그릇,(hgh2****),나도하 특: 소개할때는 최종보스급으로 나오다 나중엔 전투력측정기됨,2639,13
2,183559,199,2021-12-28 23:20,NoSignal00,(mari****),1.튜토리얼: 랩배틀 동아리 회장(래퍼)2.무학고편 흑막&천산고편 보스: '죽음에서...,1401,23
3,787411,199,2021-12-28 23:03,SSJ,(cc91****),놀랍게도 배틀만화의 정석을 밟고 있음. 반박시 니 말이 맞음.비범한 강함을 지닌 주...,1322,12
4,783052,199,2021-12-28 23:22,Djdndis,(0214****),내가볼땐 얘네 걍 쇼미나가서 우승자 줘패고 트로피 강탈하고 엔딩날듯,1110,8
...,...,...,...,...,...,...,...,...
170,783054,199,2022-01-09 23:03,유세진,(pp46****),그림체 ㅈㄴ신경쓴게 보인다,3141,23
171,758037,199,2022-01-09 23:31,rhk,(rhkr****),지수 중간에 질투하는것 같은데 언급이 없네요ㅋㅋ기린 머리 쓰다듬어 줄때 지수 밑에 ...,1979,4
172,183559,199,2022-01-09 23:55,애요옹이,(phon****),작화 좋아진게 이렇게 눈에 띌 정도는 잘 없는데 정말 신경 많이 쓰셨나봐요그림을 보...,1551,16
173,787411,199,2022-01-09 23:38,시구장구,(jw08****),몽상시리즈 스포아닌 스포하면 개쩔어요,918,23


In [ ]:
def input_text(text):
    global score
    token = tokenizing(text)
    tf = term_frequency(token)

    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data)) #새로운 데이터를 받으면 결과 예측
    if(score > 0.5):
        print("{} : 긍정 [{:.2f}%] \n".format(text, score * 100))
        score = score * 100
        
        
    else:
        print("{} : 부정 [{:.2f}%] \n".format(text, (1 - score) * 100))
        #df1['sentiment'].append(str(score))
        score = (1 - score) * 100

In [ ]:
def sentiment_print(text):
    token = tokenizing(text)
    tf = term_frequency(token)

    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data)) #새로운 데이터를 받으면 결과 예측
    if(score > 0.5):
        return score * 100
        
        
    else:
        #df1['sentiment'].append(str(score))
        return (1 - score) * 100

In [ ]:
sub_df =pd.DataFrame()

#df1['sentiment'] = 
accuracy

락커때려눕힌다음엔 발라드황제나옴? : 긍정 [82.61%] 

나도하 특: 소개할때는 최종보스급으로 나오다 나중엔 전투력측정기됨 : 긍정 [75.82%] 

1.튜토리얼: 랩배틀 동아리 회장(래퍼)2.무학고편 흑막&천산고편 보스: '죽음에서 돌아온 자' 나도하(락커) 3.한기공편 보스: '잊혀진 기계 문명의 군주' 몽키 스패너(발라드 황제) 4.말년고편 보스: '암흑군주' 계말년 (EDM) 5.네이버본사편 보스: '천국에 도달한 김준구'(Kーpop) . . . 69.성모 유니버스 총집편 보스: 'S급 갱스터 부동의 1위' THE 범차(트로트) (소년소녀의 환경과 정서에 맞게 장기연재 하기에 아무런 문제가 없는 전개입니다.) : 부정 [81.88%] 

놀랍게도 배틀만화의 정석을 밟고 있음. 반박시 니 말이 맞음.비범한 강함을 지닌 주인공 -> 무언가를 계기로 목표가 생기고 자신의 힘을 갈고 닦음 -> 그 과정에서 소소한 타이틀들을 챙김(대통령) -> 갑작스런 세계관 확장(무악공고), 더 강한 적이 나타났다(표독수) -> 기존 멤버에서 새 인물이 추가 됨(할배) 그리고 배신(융가리) -> 주인공과 쌍벽을 이룰 아치에너미 등장(락커 나도하) : 긍정 [89.25%] 

내가볼땐 얘네 걍 쇼미나가서 우승자 줘패고 트로피 강탈하고 엔딩날듯 : 부정 [99.28%] 

걱정마라 표독수!전과 4범도 대통령하겠다고 설치는 세상이다! : 부정 [56.72%] 

이제 다시 랩퍼하겠다고? 62화 동안 뭐한거냐! : 부정 [59.70%] 

대통령. 각하. 주석. 군주 ㅋㅋㅋㅋㅋㅋ그담은 뭐냐 ㅋㅋㅋ : 부정 [92.74%] 

동메달 딴거 ㅈㄴ좋으면 개추 : 긍정 [96.08%] 

나도하 : 나도 사실 락에 대해서 잘 몰라 : 긍정 [92.15%] 

고블린vs개싸이코 여고생 가슴이 웅장해진다! : 긍정 [87.56%] 

한예리 지도 예전에 같은 과였다고 다 간파하네 ㅋㅋ : 긍정 [64.44%] 

나쁜 놈들이 더 나쁜 놈을 만났다 ㄷㄷ : 부정 [55.27%] 

한예리


브릣니 이러는거 개웃기네 진짜 젊은 호빠 남자들한테 목 매는 돈 많은 아줌마 고증 제대로임 : 부정 [50.99%] 

뭐가 맙소사야 너도 쟤 남편이랑 바람피잖아ㅋㅋㅋㅋ : 부정 [95.75%] 

진짜 정상이 없네 정상이 없어ㅋㅋㅋㅋ 호빠남한테 정신 못차리는 브릣니.. 그 호빠남 소개해주고 돈뜯는 예희.. 불륜남녀와 남의 집 도촬하는 나래엄마.. : 부정 [99.73%] 

이와중에 제임스ㅋㅋㅋ 박주연한테 꽤 진심 같아서 더 킹받네ㅋㅋㅋㅋㅋㅋㅋㅋ 취존합니다 아저씨.... : 긍정 [98.82%] 

남철이 나만 웃기냐 돈많은 아줌마한테 빈대붙어서 살긴하는데 애가 멍청해서 지 배때지만 따땃하면 별다른 꿍꿍이는 없어보임ㅋㅋㅋ 다들 열심히 파국을 향해서 달려가는데 얘는 최후가 어떻게 될지 궁금하네 ㅋㅋ : 부정 [79.26%] 

나래 엄마한테 채린맘 감시하고 도촬하라고 시킨 건 예희일 듯. 채린맘 사업까지 크게 벌렸는데 이제 나락 가는 일만 남았네... 모성애를 앞세운 기업 오너가 알고 보니 불륜녀. 주가 떨어지고 투자자들 발 빼면 빚밖에 없을 텐데 이사 갈 강남 집도, 곁에 있던 남편도, 채린이까지 사라지는 소리가 들림. 모든 걸 잃은 채린맘과 모든 걸 잃은 예희... 승자가 없는 싸움이라는 게 바로 이런 걸까. : 부정 [100.00%] 

그 누구도 응원할 수 없는 만화....아무나이겨라 : 긍정 [70.34%] 

제임스 총 찾으러 온거같은데... : 긍정 [76.03%] 

아 진짜 브릣니 낌새 느껴서 안 가고 기습으로 누구 있나 문 열어서 확인할까봐 가슴 개졸였네,,,,,, : 부정 [96.89%] 

ㅋㅋㅋ 아니 예희 그와중에 브뤽니한테 개도발적인 거 ㅈㄴ 웃기네 ㅋㅋㅋㅋㅋㅋㅋ 들킬 게 없다는 거마냥 : 부정 [97.35%] 

다들 브릣니라고 대동단결하는거 넘웃김 ㅎㅎ : 긍정 [88.31%] 

근데 저 눈치 빠른 브릣니가 과연 눈치 못 채고 저런 걸까ㅋㅋㅋㅋ 알면서도 그냥 눈 감아준 것 같음.. : 긍정 [55.23%] 

제임스는 끝까지 주

## 카카오 웹툰 (메인 테이블)

* 카카오 웹툰의 경우 URL 의 변화 없이,  
  각 요일을 누르면 웹툰 리스트가 바뀌고,  
  스크롤다운을 하면 웹툰 리스트를 더 받는 동적 페이지

![image.png](attachment:image.png)

- import

In [ ]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

### 웹툰 제목과 url크롤링

#### 함수 지정

- 페이지 다운 함수 (페이지에 맞춰서 직접 지정)

In [ ]:
def page_down():
    if week == 7:
        for c in range(0,1150):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)   # 완결작품들은 1100개 가량 됨
    else :
        for c in range(0,185):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)   # 연재중 작품은 요일당 180개 밑으로 연재중

* 메인 테이블 웹툰 정보 추출 (추천 중복 5개 제외)

In [ ]:
def publishing() :
    toon = driver.find_element(By.CLASS_NAME, 'css-1saqd06')       # 웹툰 전체테이블
    x = toon.find_element(By.CLASS_NAME, 'css-19y0ur2')            # 전체에서 url 찾기 위해 준비확인
    y = x.find_elements(By.TAG_NAME,'a')                           
    titles = toon.find_elements(By.CLASS_NAME, 'css-yd766s')       # 전체에서 각 웹툰을 찾아 제목만 찾음
    if week == 7:
        for i, title in enumerate(titles):                         # 각 요일별 연재된 만화의 이름과 갯수파악 + url 연동
            print(str(i+1),title.text, y[i].get_attribute('href')) # 완결웹툰은 추천이 뜨지않음
    else:
        for i, title in enumerate(titles[5:]):                     # 각 요일별 연재된 만화의 이름과 갯수파악 + url 연동
            print(str(i+1),title.text, y[i+5].get_attribute('href'))# 요일별 웹툰은 추천으로 중첩된 값 존재하여 버리고 실행

* 필요한 정보가 있는 요일을 클릭시켜주는 함수

In [ ]:
def day_choose():
    elem = driver.find_element(By.CLASS_NAME,'css-j268k4')        #요일전체 박스 선택
    lis = elem.find_elements(By.TAG_NAME, 'li')                   #요일박스내 요일 확인
    for li in lis:
        print(li.text, end=' ')                                   #요일당 연재 및 완결한 웹툰확인
    lis[week].click()   

* 웹툰들의 각 제목과 링크를 찾아서 DB에 저장시켜주는 함수

In [ ]:
title_list = []
href_list = []
def publishing_save(title_list, href_list) :
    toon = driver.find_element(By.CLASS_NAME, 'css-1saqd06')       # 웹툰 전체테이블
    x = toon.find_element(By.CLASS_NAME, 'css-19y0ur2')            # 전체에서 url 찾기 위해 준비확인
    y = x.find_elements(By.TAG_NAME,'a')                           
    titles = toon.find_elements(By.CLASS_NAME, 'css-yd766s')       # 전체에서 각 웹툰을 찾아 제목만 찾음
    if week == 7:
        for i, title in enumerate(titles):                         # 각 요일별 연재된 만화의 이름과 갯수파악 + url 연동
            title_list.append(title.text)
            href_list.append(y[i].get_attribute('href'))
    else:
        for i, title in enumerate(titles[5:]):                     # 각 요일별 연재된 만화의 이름과 갯수파악 + url 연동
            title_list.append(title.text)
            href_list.append(y[i+5].get_attribute('href'))
    
    return title_list, href_list

- 함수 확인

In [ ]:
driver = webdriver.Chrome()
driver.get('https://page.kakao.com/main?categoryUid=10&subCategoryUid=1002')
driver.maximize_window()

In [ ]:
week = int(input())                                               # 0~7까지 요일 및 완결 페이지 변수입력
day_choose()                                                      # 함수를 통해 진행
page_down()                                                       # 페이지 다운

7
월 화 수 목 금 토 일 완결 

In [ ]:
publishing()                                                      # 연재중 웹툰 확인

1 레이디 투 퀸 https://page.kakao.com/home?seriesId=58517187
2 악당에게 사랑을 강요받고 있습니다 https://page.kakao.com/home?seriesId=55136850
3 그녀가 공작저로 가야 했던 사정 https://page.kakao.com/home?seriesId=50242834
4 신이라 불리운 사나이[완전판] https://page.kakao.com/home?seriesId=46384316
5 천애협로 https://page.kakao.com/home?seriesId=51298446
6 외과의사 엘리제 https://page.kakao.com/home?seriesId=50297433
7 도령의 가족 https://page.kakao.com/home?seriesId=53125868
8 블랙윈터 https://page.kakao.com/home?seriesId=53007019
9 김 비서가 왜 그럴까 [완결] https://page.kakao.com/home?seriesId=48704250
10 악역이 베푸는 미덕 https://page.kakao.com/home?seriesId=54363470
11 조선 세자빈 실종사건 [완결] https://page.kakao.com/home?seriesId=49134107
12 황제의 반려 https://page.kakao.com/home?seriesId=53124501
13 밥만 잘 사주는 이상한 이사님 https://page.kakao.com/home?seriesId=57944254
14 뮈엘라의 수사관 https://page.kakao.com/home?seriesId=50099907
15 미완성 연애 컬렉션 [할리퀸] https://page.kakao.com/home?seriesId=58721159
16 극한견주 https://page.kakao.com/home?seriesId=56325534
17 아내가 돌아왔다 https://page.kakao.com/ho

137 마이 페어 풋맨 https://page.kakao.com/home?seriesId=52145920
138 묘한 인생 https://page.kakao.com/home?seriesId=53868898
139 가깝고도 먼 https://page.kakao.com/home?seriesId=56392770
140 Deep(딥) https://page.kakao.com/home?seriesId=57901501
141 황성 패왕지로 컬렉션 https://page.kakao.com/home?seriesId=56796091
142 옆집남자와 소설가와 기묘한 이야기 https://page.kakao.com/home?seriesId=52552551
143 붉은 여우 https://page.kakao.com/home?seriesId=53513637
144 욕망이라는 것에 대하여 https://page.kakao.com/home?seriesId=53201510
145 자취요리왕 https://page.kakao.com/home?seriesId=53912076
146 허영만의 커피 한잔 할까요? https://page.kakao.com/home?seriesId=58402977
147 계약 아내 컬렉션 [할리퀸] https://page.kakao.com/home?seriesId=55516407
148 슈퍼리치 https://page.kakao.com/home?seriesId=55311975
149 인기는 무언가 잘못되었다 https://page.kakao.com/home?seriesId=54912842
150 황성 천하무쌍 컬렉션 https://page.kakao.com/home?seriesId=57757939
151 폭군황제의 첫사랑 https://page.kakao.com/home?seriesId=54103714
152 염라 https://page.kakao.com/home?seriesId=52821302
153 OFF STAGE (오프 스테이지) [개정판] https://

278 살벌한 동거 https://page.kakao.com/home?seriesId=50933882
279 야설록 천외비사 컬렉션 https://page.kakao.com/home?seriesId=51644743
280 씹어 삼키다 https://page.kakao.com/home?seriesId=51407005
281 야설록 중원쌍천 컬렉션 https://page.kakao.com/home?seriesId=54875200
282 우리 집 우렁이는 https://page.kakao.com/home?seriesId=50764512
283 씨엘 https://page.kakao.com/home?seriesId=46304280
284 용의 황자님 https://page.kakao.com/home?seriesId=54804855
285 불완전 결혼 컬렉션 [할리퀸] https://page.kakao.com/home?seriesId=57345620
286 기대지 마시오 https://page.kakao.com/home?seriesId=54330128
287 애증결핍 [BL/개정판] https://page.kakao.com/home?seriesId=53545137
288 스윙 배이비 [BL] https://page.kakao.com/home?seriesId=57483983
289 롤플레잉겜만화 https://page.kakao.com/home?seriesId=57896048
290 야설록 협객신화 컬렉션 https://page.kakao.com/home?seriesId=54691310
291 사악한 신데렐라에겐 악당이 필요해 https://page.kakao.com/home?seriesId=53331198
292 완전무결하게 사로잡히다[외전 연재] https://page.kakao.com/home?seriesId=50719885
293 아는 척 하지마! https://page.kakao.com/home?seriesId=55661551
294 키스톤 로맨틱 콤비 http

421 적의 심장, 그를 가지다[BL] https://page.kakao.com/home?seriesId=47750892
422 권태 https://page.kakao.com/home?seriesId=56055012
423 우리는 매일매일 https://page.kakao.com/home?seriesId=53488000
424 1 더하기 1은 https://page.kakao.com/home?seriesId=58363255
425 고행석 악질의 전설 4탄 https://page.kakao.com/home?seriesId=57342684
426 스피넬 [BL] https://page.kakao.com/home?seriesId=54165420
427 황성 무림혈투 컬렉션 https://page.kakao.com/home?seriesId=52432449
428 위험한 결혼 거래 컬렉션 [할리퀸] https://page.kakao.com/home?seriesId=56442716
429 탐닉 https://page.kakao.com/home?seriesId=53809555
430 은수 https://page.kakao.com/home?seriesId=57476010
431 엔딩을 말하다[BL] https://page.kakao.com/home?seriesId=46773026
432 가장 평범한 이세계 https://page.kakao.com/home?seriesId=56203743
433 4인4색 무림천왕 컬렉션 https://page.kakao.com/home?seriesId=54294372
434 황제는 여기 있어 기사는 퇴근할 거야 https://page.kakao.com/home?seriesId=53505972
435 고행석 악천후 패키지 https://page.kakao.com/home?seriesId=58345874
436 숨겨진 세계 https://page.kakao.com/home?seriesId=54038956
437 침대 위 짐승본능 컬렉션 [할리퀸]

557 디어 마이 유니버스 https://page.kakao.com/home?seriesId=53794568
558 Conrad (콘라드) [BL] https://page.kakao.com/home?seriesId=52390515
559 나쁜 상사[개정판] https://page.kakao.com/home?seriesId=56208574
560 남자친구를 조심해 https://page.kakao.com/home?seriesId=50009380
561 리얼 러브 독[BL] https://page.kakao.com/home?seriesId=50820682
562 사랑이 쏘다 https://page.kakao.com/home?seriesId=57336441
563 사수생도 연애를 한다 https://page.kakao.com/home?seriesId=47833314
564 구리의 구리구리 컴백 https://page.kakao.com/home?seriesId=52757904
565 오! 순정 https://page.kakao.com/home?seriesId=51705774
566 위험한 재판(개정판) [BL] https://page.kakao.com/home?seriesId=49224020
567 절벽 위의 란 https://page.kakao.com/home?seriesId=53614804
568 야설록 지옥사신 컬렉션 https://page.kakao.com/home?seriesId=56632108
569 손톱 https://page.kakao.com/home?seriesId=54096921
570 봉신연의 완전판 https://page.kakao.com/home?seriesId=51280915
571 없다 있으니까 https://page.kakao.com/home?seriesId=51449534
572 샬롯의 편지 https://page.kakao.com/home?seriesId=52292880
573 일진짱의 남자[BL] https://page.kakao.c

697 주식회사 냐옹컴퍼니 https://page.kakao.com/home?seriesId=58214323
698 청년시장 https://page.kakao.com/home?seriesId=50392468
699 초식고등학교 https://page.kakao.com/home?seriesId=55239856
700 크리미 크리피 보이 https://page.kakao.com/home?seriesId=56705138
701 아이엠 https://page.kakao.com/home?seriesId=54918683
702 만인지상 [개정판/화별] https://page.kakao.com/home?seriesId=50760306
703 리퍼스 https://page.kakao.com/home?seriesId=56152419
704 끝에서부터 한걸음 https://page.kakao.com/home?seriesId=54324517
705 바다 건너 만나러 갑니다 https://page.kakao.com/home?seriesId=53312695
706 갤러리 페이크 https://page.kakao.com/home?seriesId=46719906
707 포기하는 연애 https://page.kakao.com/home?seriesId=51693163
708 악연의 발견 [BL] https://page.kakao.com/home?seriesId=54002875
709 무색무취[BL] https://page.kakao.com/home?seriesId=51007622
710 사랑하게 될 거야 [BL] https://page.kakao.com/home?seriesId=55810491
711 러브 인 더 헬 https://page.kakao.com/home?seriesId=52216526
712 보통남남 https://page.kakao.com/home?seriesId=57072868
713 블러디 메리지 https://page.kakao.com/home?seriesId=51882

838 기생충 https://page.kakao.com/home?seriesId=56493279
839 얼티미터 https://page.kakao.com/home?seriesId=55416655
840 미스터 K(개정판) https://page.kakao.com/home?seriesId=47231535
841 Mr 뱀파이어 길들이기 https://page.kakao.com/home?seriesId=52462812
842 진의불명 [GL] https://page.kakao.com/home?seriesId=57693902
843 뇌신과 샐러리맨 [웹툰판] https://page.kakao.com/home?seriesId=57618154
844 영혼의 요리사 https://page.kakao.com/home?seriesId=56151780
845 아가씨가 너무해! https://page.kakao.com/home?seriesId=54507198
846 트라이앵글 https://page.kakao.com/home?seriesId=58318234
847 스폰서 [BL] https://page.kakao.com/home?seriesId=53465514
848 목소리의 형태 https://page.kakao.com/home?seriesId=47843489
849 기인록 https://page.kakao.com/home?seriesId=55170135
850 유물 읽는 감정사 https://page.kakao.com/home?seriesId=54516216
851 공간의 이테르 https://page.kakao.com/home?seriesId=55854431
852 수묵화 [개정판] https://page.kakao.com/home?seriesId=55481912
853 마왕님과 오타쿠 용사 https://page.kakao.com/home?seriesId=52546945
854 귀신을 보는 남자 https://page.kakao.com/home?seriesId=523148

980 최강헌터 무한 https://page.kakao.com/home?seriesId=51643850
981 헬로 마이 안드로이드 https://page.kakao.com/home?seriesId=51761548
982 안녕, 나의 크라머 https://page.kakao.com/home?seriesId=50294929
983 변이 https://page.kakao.com/home?seriesId=49801424
984 비타민C 비타민D https://page.kakao.com/home?seriesId=56548270
985 고시원 https://page.kakao.com/home?seriesId=55393362
986 집사군은 충전 중 https://page.kakao.com/home?seriesId=53136707
987 제국의 마지막 공주 https://page.kakao.com/home?seriesId=53073290
988 어비스 : 스핀오프 https://page.kakao.com/home?seriesId=52753871
989 샴 https://page.kakao.com/home?seriesId=57538835
990 달콤포근 베이커리 https://page.kakao.com/home?seriesId=55431903
991 구미호의 간에는 특별한 것이 있다 https://page.kakao.com/home?seriesId=55330392
992 디센트 https://page.kakao.com/home?seriesId=55198882
993 오토마타 스투피 https://page.kakao.com/home?seriesId=53241582
994 갓블레스유 https://page.kakao.com/home?seriesId=52202569
995 내 눈엔 너만 보여 https://page.kakao.com/home?seriesId=51471420
996 더 타임 https://page.kakao.com/home?seriesId=54891697
997 

### 크롤링 / DB 저장

- Import 및 DB 연결

In [ ]:
import psycopg2

In [ ]:
conn_str = "host=localhost dbname=myDB user=postgres password=admin port = 5432" #myDB말고도 다른 거 치면 됨 port는 바뀔수있음
try :
    conn = psycopg2.connect(conn_str)
    print("=== 접속 성공 =====")
# except:
    # print("접속오류 !!")
except psycopg2. DatabaseError as db_err:
    print("접속오류 !!")
    print(db_err)

=== 접속 성공 =====


* 테이블 생성

In [ ]:
cur = conn.cursor() #conn = 위에 나오듯 포스트그리 커넥션으로 연결을 말한다.
try:
    cur.execute('create table if not exists KAKAO (num serial, day_of_week varchar(10),webtoon_name varchar(50), webtoon_url varchar(200));')
    conn.commit()                         #테이블 생성
    print('생성 성공')
except :
    conn.rollback()
    print('---DB error')

생성 성공


* 제목과 링크 크롤링 후 저장

In [ ]:
saving=publishing_save(title_list, href_list)       # 제목과 url을 리스트화한 후 확인
saving

In [ ]:
for i,title in enumerate(title_list):
    try :
        cur. execute('INSERT INTO KAKAO (num,day_of_week,webtoon_name,webtoon_url) VALUES (%s, %s, %s, %s);',
        ((i+1),'완결',title, href_list[i]))                         # 4개의 column에 번호와 연재일,제목,url추가
        conn.commit()
        print("----- insert ok--")                                     
    except :
        conn. rollback()
        print("--- DB error")

        
title_list = []                                                      #모두 진행후 리스트 내용 제거
href_list = []

----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- inse

----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- inse

----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--


![image.png](attachment:image.png)

## 카카오 웹툰 (댓글 테이블)

### 각 웹툰의 화 수와 각 화당 달린 댓글확인

In [ ]:
# 링크가 클릭이 되지않아 일일히 들어가는 것으로 대체하였음

body = driver.find_element(By.TAG_NAME,'body')                   #각 화당 적힌 댓글 이걸 drive말고 움직인 사이트로 지정해야함
reples = body.find_elements(By.CLASS_NAME,'css-av465m')          #웹툰 각 화에서 실행시 보여짐
chapter = body.find_element(By.CLASS_NAME,'css-4mn7vf')          #웹툰 각 화 수
for c in range(0,1500):                                          #range는 전체댓글에 따라 다르게 적어줘야함
    driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)

for i,reple in enumerate(reples): 
    reple_time = body.find_elements(By.CLASS_NAME,'css-xy7wav')  #리플단 날짜
    print(chapter.text,reple_time[i].text,reple.text)            # 리플 내용과 단 날짜

황제의 아이를 숨기는 방법 1화 2022.02.03 그림체가 하네되랑 너무 닮아서 왜 그런지 보니까 하네되 작화 도움 맡으시는 분이 그림 작가시네요!! 그림 너무 예뻐요 응원할게요 😍😍
황제의 아이를 숨기는 방법 1화 2022.02.03 벌써 후회남의 싹이 보인다..아아...
그는 건너버렸어요..후회의 강..
황제의 아이를 숨기는 방법 1화 17분전 이제와서 찾은 애비가 뭔 소용이야
황제의 아이를 숨기는 방법 1화 6시간전 남주최악이고 내용도 고구만데 그림체가 너무좋아...  그냥 기무로 내용은 생각하지 말고 그림만 감상해야겠다..ㅎㅎ
황제의 아이를 숨기는 방법 1화 9시간전 남주 최악인데 이거
황제의 아이를 숨기는 방법 1화 9시간전 소설 재밌게 본 기억이 있네요ㅎㅎ 잘 부탁드려요:)
황제의 아이를 숨기는 방법 1화 10시간전 아  소설이 웹툰으로 좋아욤^^
황제의 아이를 숨기는 방법 1화 12시간전 작화가  넘 예뻐요
응원합니다 
황제의 아이를 숨기는 방법 1화 12시간전 나쁜놈~~이혼하자해놓고 왜찾어? 하는짓이 이번 저질 동올 중국놈들이네
황제의 아이를 숨기는 방법 1화 13시간전 팔 거 하나 생겼다 신작 중에 긂체 괜찮은 거 건졌네
황제의 아이를 숨기는 방법 1화 14시간전 와 그림 너무 잘그린다 취향저격당함
황제의 아이를 숨기는 방법 1화 14시간전 먼저 버렸으면서.. 왜 찾고 난리야💢
황제의 아이를 숨기는 방법 1화 17시간전 와 열심히 읽은 원작이 웹툰으로 나온걸 보네요 감사합니다
황제의 아이를 숨기는 방법 1화 21시간전 후회남 데굴데굴 덱데굴..10년에 아이 육아6년까지 이렇게 오랜세월 고생시키다니 평생 회개해도 모자랄듯..작가님 남주를 원통인듯 공인듯 바퀴인듯 맘껏 굴려주세요!
황제의 아이를 숨기는 방법 1화 23시간전 그림체 쥑이네
황제의 아이를 숨기는 방법 1화 2022.02.08 잘 볼게요.응원합니다!!~
황제의 아이를 숨기는 방법 1화 2022.02.08 어릴적에 그렇게 웃었으면서...니가 꼬셨잖아 ㅠㅠㅠ
황제의 아이를 숨기는 방법 1

황제의 아이를 숨기는 방법 1화 2022.02.05 그림체 하네되랑 되게 비슷하네요!!
황제의 아이를 숨기는 방법 1화 2022.02.05 처음에는 4화꺼지 보고 천천히 기무로 봐야지 하는 마음읋 시작... 소설 131화까지... 쭈우욱... 다녀옴... ㅎㅎ 웹툰 보고 소설 넘어가니 멋진 그림 덕분에 더욱 더 즐거움요😍
황제의 아이를 숨기는 방법 1화 2022.02.05 응원합니다 
황제의 아이를 숨기는 방법 1화 2022.02.05 응원합니다 
황제의 아이를 숨기는 방법 1화 2022.02.05 반갑습니다 !!
웾툰으로 만나네요 
황제의 아이를 숨기는 방법 1화 2022.02.05 응원합니다
황제의 아이를 숨기는 방법 1화 2022.02.05 찾지마!!!! 찾지말라고!!!!.....
황제의 아이를 숨기는 방법 1화 2022.02.05 이작품을 웹툰으로 보니 새롭다
황제의 아이를 숨기는 방법 1화 2022.02.05 황제 ♩♪♬♩ !!!
황제의 아이를 숨기는 방법 1화 2022.02.05 이웃집웹툰이랑 더불어 막장드라마같은 재미를 주겠구먼 후후후~
황제의 아이를 숨기는 방법 1화 2022.02.05 와 작화 대박..
황제의 아이를 숨기는 방법 1화 2022.02.05 하루만에 이혼할땐 언제고 이제와서 찾냐?
황제의 아이를 숨기는 방법 1화 2022.02.05 소설로봣다 글읽는게 싫어서 때려쳤는데 웹툰으로 나왓네??????
황제의 아이를 숨기는 방법 1화 2022.02.05 귀족 숙청할 때 위험할까봐 속인거 같은데...
황제의 아이를 숨기는 방법 1화 2022.02.05 작화 뭐래요?넘 예뻐요!!!
황제의 아이를 숨기는 방법 1화 2022.02.05 응원합니다 
황제의 아이를 숨기는 방법 1화 2022.02.05 누가봐도 그 ㅅㄲ 아들이네..ㅜㅜ슬프다
황제의 아이를 숨기는 방법 1화 2022.02.05 왜 찾아
꺼져 !
황제의 아이를 숨기는 방법 1화 2022.02.05 재밌게 읽은 소설이 멋진
그림체로 옷을입고 나왔네요.
스토리 좋고 만화 좋아하는 나의 눈을

황제의 아이를 숨기는 방법 1화 2022.02.04 오그림체 합격
황제의 아이를 숨기는 방법 1화 2022.02.04 하네되 그림체 너무 비슷하네요 아무리 어시분이었대도
황제의 아이를 숨기는 방법 1화 2022.02.04 주둥이 주둥이 때려주고싶네
황제의 아이를 숨기는 방법 1화 2022.02.04 다시.. 다시.. 다시 처음부터 볼거야 ㅠㅠ 계속 볼거야 ㅠㅠ 소설도 볼거야 ㅠㅠㅠㅠㅠㅠ 애착바디필로우 주세요!! 그거라도 주세요!! 다음화 바로 안 주실거면 ㅠㅠㅠㅠㅠㅠㅠ
황제의 아이를 숨기는 방법 1화 2022.02.04 작화 실화?
황제의 아이를 숨기는 방법 1화 2022.02.04 왜찾냐고어이없어
황제의 아이를 숨기는 방법 1화 2022.02.04 응원응원합니다 화이팅하세요 
황제의 아이를 숨기는 방법 1화 2022.02.04 응원합니다 
황제의 아이를 숨기는 방법 1화 2022.02.04 
황제의 아이를 숨기는 방법 1화 2022.02.04 와 미쳤다 벌써 흥미진진함..ㄷㄷ
황제의 아이를 숨기는 방법 1화 2022.02.04 아 뭐야 벌써부터 재미있을것 같아... 한 50편쯤 나왔을때까지 기다렸다가 볼까 했는데ㅠㅠ 지금 바로 달린다....
황제의 아이를 숨기는 방법 1화 2022.02.04 테오르.. 너무 예쁘게 생겼다...
황제의 아이를 숨기는 방법 1화 2022.02.04 세상에...이거 예전에 소설로 조금 봤었는데... 그림체가 너무 예뻐요ㅠㅠㅠ
황제의 아이를 숨기는 방법 1화 2022.02.04 와 대박 ㅠㅠㅠㅠㅠㅠㅠㅠ
황제의 아이를 숨기는 방법 1화 2022.02.04 사랑하지 않았다던 태자와 하룻밤을 잤더니 애가 생겼다? 이로써 손 잡고 자면 애가 생긴다는 이야기는 진실인 것으로...
황제의 아이를 숨기는 방법 1화 2022.02.04 70p 테오르 너무 기엽다..
황제의 아이를 숨기는 방법 1화 2022.02.04 소설 남았는더 그냥 웹툰보고 싶네요. 테오르 너무 멋지고 귀엽고 딱 상상했던 아이라서 작가님께 큰절해야 할듯요
황제의 아이를 숨기는

황제의 아이를 숨기는 방법 1화 2022.02.04 아잇씨ㅜㅠ버려놓고 후회하지 말라고!...ㅠㅜㅜ
황제의 아이를 숨기는 방법 1화 2022.02.04 헐~~ 너무 귀여워요
황제의 아이를 숨기는 방법 1화 2022.02.04 지가 그렇게 버려놓고 이제와서;;
황제의 아이를 숨기는 방법 1화 2022.02.04 소설보다 재미있음 좋겠네요. 정말 징글징글했던건데....
황제의 아이를 숨기는 방법 1화 2022.02.04 다시봐도 우리 아스텔 너무 눈물나...
황제의 아이를 숨기는 방법 1화 2022.02.04 하룻밤임신이면ㄷㄷ남주이자식좀치는데?
황제의 아이를 숨기는 방법 1화 2022.02.04 응원합니다.
황제의 아이를 숨기는 방법 1화 2022.02.04 악 이거 소설로도 넘나 잼나게 봐서 재탕까지 했는데 웹툰으로 나왔다니요 😢 
황제의 아이를 숨기는 방법 1화 2022.02.04 와~~그림체가 너무 예뻐서 기대됩니다!
황제의 아이를 숨기는 방법 1화 2022.02.04 오그림체 합격
황제의 아이를 숨기는 방법 1화 2022.02.04 하네되 그림체 너무 비슷하네요 아무리 어시분이었대도
황제의 아이를 숨기는 방법 1화 2022.02.04 주둥이 주둥이 때려주고싶네
황제의 아이를 숨기는 방법 1화 2022.02.04 다시.. 다시.. 다시 처음부터 볼거야 ㅠㅠ 계속 볼거야 ㅠㅠ 소설도 볼거야 ㅠㅠㅠㅠㅠㅠ 애착바디필로우 주세요!! 그거라도 주세요!! 다음화 바로 안 주실거면 ㅠㅠㅠㅠㅠㅠㅠ
황제의 아이를 숨기는 방법 1화 2022.02.04 작화 실화?
황제의 아이를 숨기는 방법 1화 2022.02.04 왜찾냐고어이없어
황제의 아이를 숨기는 방법 1화 2022.02.04 응원응원합니다 화이팅하세요 
황제의 아이를 숨기는 방법 1화 2022.02.04 응원합니다 
황제의 아이를 숨기는 방법 1화 2022.02.04 
황제의 아이를 숨기는 방법 1화 2022.02.04 와 미쳤다 벌써 흥미진진함..ㄷㄷ
황제의 아이를 숨기는 방법 1화 2022.02.04 아 뭐야 벌써부터

황제의 아이를 숨기는 방법 1화 2022.02.04 황제 전남편 🐕 시키!!!이제와서 찾는건 뭐야!!!1818181818
황제의 아이를 숨기는 방법 1화 2022.02.04 자 지를 시간입니다
황제의 아이를 숨기는 방법 1화 2022.02.04 오 대작 느낌..
황제의 아이를 숨기는 방법 1화 2022.02.04 오.계속 보고프네요
황제의 아이를 숨기는 방법 1화 2022.02.04 오~~소설을  읽을것을  웹툰으로  나왔네요 재미있게 읽을게요 너무좋네요 ㅎ그림체도  예쁘네요~~♡
황제의 아이를 숨기는 방법 1화 2022.02.04 대충 못숨기겠군..
황제의 아이를 숨기는 방법 1화 2022.02.04 와 ㅠㅠ 이번에 제가 좋아하는 거 다 시즌 끝나서 우울했는데 올만에 탄탄한 스토리 로판 웹툰 나와서 가슴이 두근두근!!!
황제의 아이를 숨기는 방법 1화 2022.02.04 시작부터 남주굴리기가 기대되는 웹툰은 처음...
황제의 아이를 숨기는 방법 1화 2022.02.04 그림체 예뻐요
황제의 아이를 숨기는 방법 1화 2022.02.04 애기 너무 잘생겼어♡♡
황제의 아이를 숨기는 방법 1화 2022.02.04 그림체가 너무 예쁘네요.
눈동자색 변하고 있는것도 잘표현 되어있어요
황제의 아이를 숨기는 방법 1화 2022.02.04 이정도면 후회남주 정도로는 어림없을것 같은데요?
구르고 굴러야 할것 같아요
황제의 아이를 숨기는 방법 1화 2022.02.04 미친ㄱ새낀가 하루 먹버하고 이제와서 쫄래쫄래 찾아다님? 지새끼라고? ♬~♪♬ㅇ
황제의 아이를 숨기는 방법 1화 2022.02.04 와 작가님 연출이 대박이세요 소설 기억안날정도! 소설은 끝까지 못봤는데 웹툰은 기대만땅입니다♡
황제의 아이를 숨기는 방법 1화 2022.02.04 남주....어쩔...무릎이  곧 닳고  닳아지겠구나....
너는....진짜  많이  굴러야겠다....애가6살이니...임신기간 포함7년만  빡시게 구르쟈...
황제의 아이를 숨기는 방법 1화 2022.02.04 응원해요!
남주 이혼하자고 먼저

황제의 아이를 숨기는 방법 1화 2022.02.04 테오 존잘~~~~~♡♡♡♡
황제의 아이를 숨기는 방법 1화 2022.02.04 이쁜 테오르..
황제의 아이를 숨기는 방법 1화 2022.02.04 오///재밌겠는데
황제의 아이를 숨기는 방법 1화 2022.02.04 .???????????????????? 사랑..합니다..
황제의 아이를 숨기는 방법 1화 2022.02.04 저리 가 ㅠㅠ 이제와서 어쩌라고 개ㅐ자식아ㅏ
황제의 아이를 숨기는 방법 1화 2022.02.04 하네되 느낌 좋아요~~ 다만 눈이 좀 가운데로 안 몰리게 해주세요ㅜㅜ
황제의 아이를 숨기는 방법 1화 2022.02.04 응원합니다 
황제의 아이를 숨기는 방법 1화 2022.02.04 킁킁 명작의 냄새가 난다! 그림이 너무 예뻐요 응원해요!_!
황제의 아이를 숨기는 방법 1화 2022.02.04 버릴 땐 언제고 이제와서 다시 찾냐 황제 10 8 새끼
황제의 아이를 숨기는 방법 1화 2022.02.04 10년을 개고생시키고 하룻밤만에 황제된다고 쪼차내다니....ㄱㅐ쉐이네???
황제의 아이를 숨기는 방법 1화 2022.02.04 아닛 하룻밤 사이에 이혼할 거면 안 해야할 짓은 왜 했냐
황제의 아이를 숨기는 방법 1화 2022.02.04 아니 먹버야??이혼할거면 안자야지ㅋㅋㅋ 나촴!!
황제의 아이를 숨기는 방법 1화 2022.02.04 그림체 미쵸
황제의 아이를 숨기는 방법 1화 2022.02.04 뭐야…..그렇게  싫으면서 하룻밤은  왜자고  왜  찾아와?……xx ccx xxx cxx xxx
황제의 아이를 숨기는 방법 1화 2022.02.04 그림도 스토리도 좋아요~^^
응원합니다 
황제의 아이를 숨기는 방법 1화 2022.02.04 응원합니다
황제의 아이를 숨기는 방법 1화 2022.02.04 작가님  작화 너무 시원시원하고 넘 예뻐요. 저는 하네되...하고는 다르다고 생각됩니다~작가님만의 고유한 작화색이 있는거니깐 ♡♡♡♡♡
황제의 아이를 숨기는 방법 1화 2022.02.04 힣 남주 구르는것

황제의 아이를 숨기는 방법 1화 2022.02.04 황제 ♪♩♩~ㅏ라마 니가 아무리 흑발적안이어도 못 봐준다
지구 열바퀴 구르고 와라
황제의 아이를 숨기는 방법 1화 2022.02.04 아 헐 대박 어쩐지 여주 분위기가 어디서 본거같다 햇더니!!
황제의 아이를 숨기는 방법 1화 2022.02.04 하네되 작가님의 또 다른 작품 ㅎㄷㄷㄷ
황제의 아이를 숨기는 방법 1화 2022.02.04 웹툰 축하해요. 소설도 재미있게 읽었는데 웹툰 나와서 넘 기뻐요 ㅎㅎ
황제의 아이를 숨기는 방법 1화 2022.02.04 엄마야 깜짝이야...!
황제의 아이를 숨기는 방법 1화 2022.02.04 ㅡㅡ왜찾고♩♬♬이야
와씨8노마!!!!
황제의 아이를 숨기는 방법 1화 2022.02.04 황제놈 다른 여자랑  애를 못 낳았나봐 ㅠ ㅠ
황제의 아이를 숨기는 방법 1화 2022.02.04 아이게웹툰화라니ㅠㅠㅠㅜ미쳤나봐사랑해요진짜ㅠㅜㅠ
황제의 아이를 숨기는 방법 1화 2022.02.04 처음엔 100캐시 얻으려고 왔는데 보다보니 너무 재밌다,,
황제의 아이를 숨기는 방법 1화 2022.02.04 와우!작화 넘 이쁘네요. 스토리진행도 쳐낼거 팍팍쳐내고 핵심만 나오니 좋고요ㅋㅋㅋ 정주행해야겠어요.
황제의 아이를 숨기는 방법 1화 2022.02.04 응원합니다
황제의 아이를 숨기는 방법 1화 2022.02.04 대작이였어 와우~~~~
황제의 아이를 숨기는 방법 1화 2022.02.04 하네되 생각났는데 베댓보고 알게됐음ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 넘 재밌다 저 애 너무 내 취향임 ㅠㅠㅋㅋㅋ
황제의 아이를 숨기는 방법 1화 2022.02.04 와씨 이게 웹툰으로. .
황제의 아이를 숨기는 방법 1화 2022.02.04 으어 이 작품이 웹툰으로 나오다니..
황제의 아이를 숨기는 방법 1화 2022.02.04 소설로 재밌게 봤던 소설이 웹툰으로 나오다니!!!! 최고입니다!!!
황제의 아이를 숨기는 방법 1화 2022.02.04 그림 너무 고급져서 깜짝 
황제의 아이를 숨기는 방법 1화 2022.02.04

황제의 아이를 숨기는 방법 1화 2022.02.04 버려놓구 찾는 심보는 뭐람 폐하 굴러라 굴럿!!!!!
황제의 아이를 숨기는 방법 1화 2022.02.04 오오~이게 웹툰으로 나와주다니!!!
심지어 작화 고퀄 굿입니다!
황제의 아이를 숨기는 방법 1화 2022.02.04 와@@@@
테오르
소설로 보다 만나니 또
이렇게 반갑구나
아우 귀여워♡♡
황제의 아이를 숨기는 방법 1화 2022.02.04 작화가 미쳤네요 
황제의 아이를 숨기는 방법 1화 2022.02.04 이벤트참여 
황제의 아이를 숨기는 방법 1화 2022.02.04 원작 전개가 고구마긴 했지만 나름 잼있게 봤는데!!
이게 웹툰이 되다니ㅠㅠㅠㅠ
황제의 아이를 숨기는 방법 1화 2022.02.04 지금부터 똥이 후회하고 남주가 되는 과정을 보여주는 건가
황제의 아이를 숨기는 방법 1화 2022.02.04 남주 쓰레기란 말이 많네 작가님이 각색좀 해서 올려줬으면 제대로 굴러서 후회남을 보여주세요!!
황제의 아이를 숨기는 방법 1화 2022.02.04 원작 평이 그렇게 좋지 않았던걸로 기억하는데 이건... 이건... 작화 때문이라도 부여잡고 봐야겠어요... 최고다 진짜...
황제의 아이를 숨기는 방법 1화 2022.02.04 하.... 똥이네 그런데 그똥을 다시 남주로 만드는거?
황제의 아이를 숨기는 방법 1화 2022.02.04 아니 근데 딱히 결혼하고싶지도 않았던 여자랑 초야는 왜 치름? 피곤하다 핑계대고 걍 등돌리고 자지
황제의 아이를 숨기는 방법 1화 2022.02.04 남주가.. 후회남인데..
마음만 후회만 하고 구르지는 않았다해서.. 시작을 안해야겠음!!
.
근데 그림 맘이 들고요 ㅠㅠ 큭
황제의 아이를 숨기는 방법 1화 2022.02.04 눈 색 바꾸는거 보고 표지 다시 한번 보고왔는데 머리도 눈색도 생물학적 아빠를 닮아서 어떻게든 부정하려고 눈색을 바꿨나보네요.
진짜진짜 맗이 굴렀으면ㅂㄷㅂㄷ
황제의 아이를 숨기는 방법 1화 2022.02.04 남주 개쓰레긴 그럴거면 하기전에 이혼하자고하던가 

황제의 아이를 숨기는 방법 1화 2022.02.04 돈 나가는소리가 또 들려온다 ㅠㅠㅋㅋ
황제의 아이를 숨기는 방법 1화 2022.02.04 정주행 시작입니다...
황제의 아이를 숨기는 방법 1화 2022.02.04 헉 테오르 아가 너무 예쁘다...
황제의 아이를 숨기는 방법 1화 2022.02.04 
황제의 아이를 숨기는 방법 1화 2022.02.04 황제 개빡치네 벌써
황제의 아이를 숨기는 방법 1화 2022.02.04 헐 이 소설 진짜 재밌게 읽었는데 완전 응원합니ㅡㅏ
황제의 아이를 숨기는 방법 1화 2022.02.04 황제페하 왜 이제  ㅈ찾으세요ㅡㅡ(╬ ಠ 益ಠ)
황제의 아이를 숨기는 방법 1화 2022.02.04 나 이거 틱톡에서 보고 나서 소설 다 질렀고 2월달에 웹툰나오는거 알고 기다렸는데!!! ㅠㅠ 심지어 하네되 작화 도움 맡으시는 분이라니ㅠㅠ 
황제의 아이를 숨기는 방법 1화 2022.02.04 황제야 굴러라 굴러 굴렁쇠가 되어라
황제의 아이를 숨기는 방법 1화 2022.02.04 와...그림체땜에 빠져 들어왔다 ㄷㄷ
황제의 아이를 숨기는 방법 1화 2022.02.04 원작 읽었는데 스포하고 싶다..
황제의 아이를 숨기는 방법 1화 2022.02.04 곰인형 안고있는거 너무 귀엽다,,,♡
황제의 아이를 숨기는 방법 1화 2022.02.04 아주써글노믜새기구나? ? 남주는 아니길 바란다 정말..
황제의 아이를 숨기는 방법 1화 2022.02.04 로판엔 왜 ㄱh 슈렉이 남주가 넘쳐나는걸까..??
아무리 후회남클리셰라고해도 저런 가습기같은 것들이 로판남주조연 7대1 비율차지하는건 넘하지않나?? (나머지 2는 내아이에게만 따뜻한 아바님)
첫등장씬 넘기기도전에 욕부터나오네 진짜.

루카스 디트리안봐 얼마나 좋아. 니들이 진정 찐이고 빛이며 남주의정석이다.
황제의 아이를 숨기는 방법 1화 2022.02.04 와씨 와...와.....진짜....내 인생 소설이 ...와...
황제의 아이를 숨기는 방법 1화 2022.02.04 이혼했는데 왜 찾어
황제의 아

* 저장 함수 지정

In [ ]:
chapter_list = []
day_list = []
reple_list=[]

def reple_save(chapter_list,day_list,reple_list) :
    body = driver.find_element(By.TAG_NAME,'body')                   #각 화당 적힌 댓글 이걸 drive말고 움직인 사이트로 지정해야함
    reples = body.find_elements(By.CLASS_NAME,'css-av465m')          #웹툰 각 화에서 실행시 보여짐
    chapter = body.find_element(By.CLASS_NAME,'css-4mn7vf')          #웹툰 각 화 수
    
    for i,reple in enumerate(reples): 
        reple_time = body.find_elements(By.CLASS_NAME,'css-xy7wav')  #리플단 날짜
        chapter_list.append(chapter.text)
        day_list.append(reple_time[i].text)                    #리스트에 내용들 저장
        reple_list.append(reple.text)
    return chapter_list,day_list,reple_list                   # 리플 내용과 단 날짜


* DB 생성 및 저장

In [ ]:
try:
    cur.execute('create table if not exists KAKAO_reple (num serial, chapter varchar(50), day varchar(50), reple varchar(500));')
    conn.commit()
    print('생성 성공')
except :
    conn.rollback()
    print('---DB error')

In [ ]:
reple_save(chapter_list,day_list,reple_list)            #웹툰의 화, 날짜, 댓글내용 리스트에 저장

(['황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 숨기는 방법 1화',
  '황제의 아이를 

In [ ]:
for i,chapter in enumerate(chapter_list):
    try :
        cur. execute('INSERT INTO KAKAO_reple (num,chapter,day,reple) VALUES (%s, %s, %s, %s);',
        ((i+1),chapter,day_list[i],reple_list[i]))
        conn.commit()                                                   # 댓글창과 날짜 댓글이 달린 화 표시
        print("----- insert ok--")                                      # 4개의 column에 번호와 연재일,제목,url추가
    except :
        conn. rollback()
        print("--- DB error")

----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- inse

----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- insert ok--
----- inse

![image.png](attachment:image.png)

### 댓글 csv저장

In [ ]:
import pandas as pd

* 데이터 프레임화

In [ ]:
data = {'chapter':chapter_list,'day':day_list,'reple':reple_list} 
df = pd.DataFrame(data)
df

,chapter,day,reple
0,황제의 아이를 숨기는 방법 1화,2022.02.03,그림체가 하네되랑 너무 닮아서 왜 그런지 보니까 하네되 작화 도움 맡으시는 분이 그...
1,황제의 아이를 숨기는 방법 1화,2022.02.03,벌써 후회남의 싹이 보인다..아아...\n그는 건너버렸어요..후회의 강..
2,황제의 아이를 숨기는 방법 1화,17분전,이제와서 찾은 애비가 뭔 소용이야
3,황제의 아이를 숨기는 방법 1화,6시간전,남주최악이고 내용도 고구만데 그림체가 너무좋아... 그냥 기무로 내용은 생각하지 ...
4,황제의 아이를 숨기는 방법 1화,9시간전,남주 최악인데 이거
...,...,...,...
1220,황제의 아이를 숨기는 방법 1화,2022.02.03,작화 미친.
1221,황제의 아이를 숨기는 방법 1화,2022.02.03,테오르으으-!!!
1222,황제의 아이를 숨기는 방법 1화,2022.02.03,후회남이기만 해봐 진짜 가만안둠
1223,황제의 아이를 숨기는 방법 1화,2022.02.03,카이젠 머리 박고 후회하면서 디스코 주나요? 그때까지 힘내서 응원하겠습니다


In [ ]:
df.to_csv("./kakao_reple.csv", index= False , encoding= 'utf-8-sig')      #데이터 csv로 저장, utf-8-sig로 해야 한글이 꺠지지 않는다.